In [1]:
# 이전년도 감자 작물 하나에 대한 데이터 통합
import xlrd
from datetime import datetime
import os, shutil
from nltk.tokenize import word_tokenize
import cx_Oracle
import numpy as np
import pandas as pd
from datetime import datetime
from openpyxl import load_workbook
from dateutil.relativedelta import relativedelta
import csv
import re

now = datetime.now()

today_str = str(now)
today_day_and_time = []
today_day_and_time = word_tokenize(today_str)
today_day = today_day_and_time[0].split('-')

year = today_day[0]  # 현재년
month = today_day[1]  # 현재월
day = today_day[2]  # 현재일

before_one_year = now - relativedelta(years=1)

before_today_str = str(before_one_year)
before_day_and_time = []
before_day_and_time = word_tokenize(before_today_str)
beforey_day = before_day_and_time[0].split('-')

before_year = beforey_day[0]  # 저번년
before_month = beforey_day[1]  # 저번월
before_day = beforey_day[2]  # 저번일

# LOCATION = r"C:\\instantclient-basic-windows.x64-19.11.0.0.0dbru\\instantclient_19_11" # 여러분들의 instantclient 가 있는 경로, \표시는 두개로 써줘야함
# conn = cx_Oracle.connect("waap", "12345", "localhost:1521/xe") # 여러분들의 접속 이름, 비밀번호, 호스트이름, 포트, sid


###################################################################################################################################
# 날짜를 비교하기 위한 기상데이터

f1 = open(before_year + '대전기온.csv')  # 이전년도 기온.csv
temperature_data1 = csv.reader(f1)
header = next(temperature_data1)
cnt = 0
temperature_realdata1 = []
for row in temperature_data1:
    cnt += 1
    if (cnt > 11):
        temperature_realdata1.append(row)

        # 데이터에서 \t 문자열 제거
        # -------------------------------------------------

        characters = "\t"
        try:
            string = row[0]
        except IndexError:
            continue
        for x in range(len(characters)):
            # string = string.replace(characters[x],"")# \t 삭제한 row[0] 을 string 변수에 저장
            string = "0"
            cnt_list = cnt - 12  # temperature_realdata1[0][0]을 자동으로 카운트해주기 위해

            temperature_realdata1[cnt_list][0] = string  # temperature_realdata1[n][0]에 있는 리스트 요소를 string으로 바꾸기 위함
f1.close()
# __________________________________________________

# 필요한 타이틀만 남겨두기
# -------------------------------------------------
cnt_1 = 0

for p in temperature_realdata1:

    try:
        del temperature_realdata1[cnt_1][5]

    except IndexError:
        continue
    cnt_1 += 1

cnt_1 = 0
cnt_2 = 7

for p in temperature_realdata1:
    for x in range(2):
        try:
            del temperature_realdata1[cnt_1][cnt_2]

        except IndexError:
            continue

        cnt_2 -= 1
    cnt_2 = 7
    cnt_1 += 1

cnt_1 = 0

for p in temperature_realdata1:

    try:
        del temperature_realdata1[cnt_1][0]

    except IndexError:
        continue
    cnt_1 += 1

# __________________________________________________

# 저장된 데이터에서 비어있는 쓰레기값 제거
# -------------------------------------------------
lendata = len(temperature_realdata1) - 11

for p1 in range(11):
    try:
        del temperature_realdata1[lendata]
    except IndexError:
        continue
i = 0
k = 0
datelist = [0 for row in range(len(temperature_realdata1))]
for i in range(len(temperature_realdata1)):
    datelist[i] = temperature_realdata1[i][1]

###################################################################################################################################


procnt = 0

kindcode = ['쌀', '콩', '팥', '고구마', '감자', '시금치', '열무', '대파', ]
unitcode = ['']
colslist = [[0 for col in range(84)] for row in range(1)]

for pro in range(len(kindcode)):
    print(kindcode[pro])
    procnt += 1

    try:
        df = pd.read_excel('농산물 ' + before_year + kindcode[pro] + '.xlsx', header=0)
    except:
        continue
    if kindcode[pro] == '쌀':
        print(kindcode[pro], '  데이터 정제 시작')

        df_list = df.values.tolist()
        df_col = list([col for col in df])
        del df_col[0]

        pro_date = [0 for i in range(len(df_list[0]) - 1)]
        avg = [0 for i in range(len(df_list[0]) - 1)]  # 전체 평균값
        avg_fake = [0 for i in range(len(df_list[0]) - 2)]
        seoul = [0 for i in range(len(df_list[0]) - 1)]  # 서울 평균값
        busan = [0 for i in range(len(df_list[0]) - 1)]  # 부산 평균값
        daegoo = [0 for i in range(len(df_list[0]) - 1)]  # 대구 평균값
        goangjoo = [0 for i in range(len(df_list[0]) - 1)]  # 광주 평균값
        daejeon = [0 for i in range(len(df_list[0]) - 1)]  # 대전 평균값

        # --------------------------------------------------------데이터상의 날짜 구하기
        for i in range(len(df_col)):
            pp = word_tokenize(str(df_col[i]))
            z = pp[0]
            try:

                pro_date[i] = z
            except:
                continue
        i = 0

        for k in range(len(pro_date)):
            pro_date[k] = pro_date[k].replace('2021', '2020')

        # --------------------------------------------------------

        # ========================================================평균가격,서울,부산,대구,광주,대전 별로 리스트 분리

        for i in range(len(avg)):
            if i == 0:
                try:
                    avg = df_list[i][1:-1]
                except:
                    continue
            elif i == 1:
                continue

            elif i == 2:
                try:
                    seoul = df_list[i][1:]
                except:
                    continue
            elif i == 3:
                try:
                    busan = df_list[i][1:]
                except:
                    continue

            elif i == 4:
                try:
                    daegoo = df_list[i][1:]
                except:
                    continue
            elif i == 5:
                try:
                    goangjoo = df_list[i][1:]
                except:
                    continue
            elif i == 6:
                try:
                    daejeon = df_list[i][1:]
                except:
                    continue

        # ========================================================

        location = {'서울': 'seoul', '부산': 'busan', '대구': 'daegoo', '광주': 'goangjoo', '대전': 'daejeon'}
        pro_duct1 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct2 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct3 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct4 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct5 = [[0 for col in range(4)] for row in range(len(pro_date))]

        fakelist = [0 for row in range(len(datelist))]
        for i in range(len(pro_date)):
            fakelist[i] = pro_date[i]

        lost_date = list(set(datelist) - set(fakelist))

        sum_fake_list1 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list2 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list3 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list4 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list5 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        i = 0

        # -------------------------------------------------------------------평균가의 평균가격을 구함
        for i in range(len(avg)):
            value = avg[i].replace(',', '')
            avg[i] = int(value)

        for fake in range(len(avg_fake)):
            try:
                avg_fake[fake] = avg[fake]
            except:
                continue

        i = 0

        for i in range(len(avg_fake)):
            try:
                avg_fake[i] += avg_fake[i - 1]
                if i == (len(avg) - 1):
                    avg_fake[i] = avg_fake[i] - avg[i]
            except:
                continue

        avg_plus_avg = int((avg_fake[(len(avg_fake) - 1)]) / len(avg_fake))

        for lo in location:

            i = 0

            if lo == '서울':

                for i in range(len(seoul)):  # 서울 가격의 문자열 제거후 int로 변경
                    if seoul[i] == '-':
                        seoul[i] = seoul[i].replace('-', str(avg_plus_avg))
                    elif seoul[i] == '0':
                        seoul[i] = seoul[i].replace('0', str(avg_plus_avg))

                    value = re.sub(',', '', seoul[i])
                    seoul[i] = int(value)



            elif lo == '부산':

                for i in range(len(busan)):  # 부산 가격의 문자열 제거후 int로 변경
                    if busan[i] == '-':
                        busan[i] = busan[i].replace('-', str(avg_plus_avg))
                    elif busan[i] == '0':
                        busan[i] = busan[i].replace('0', str(avg_plus_avg))

                    value = re.sub(',', '', busan[i])
                    busan[i] = int(value)

            elif lo == '대구':

                for i in range(len(daegoo)):  # 대구 가격의 문자열 제거후 int로 변경
                    if daegoo[i] == '-':
                        daegoo[i] = daegoo[i].replace('-', str(avg_plus_avg))
                    elif daegoo[i] == '0':
                        daegoo[i] = daegoo[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', daegoo[i])
                    daegoo[i] = int(value)

            elif lo == '광주':

                for i in range(len(goangjoo)):  # 광주 가격의 문자열 제거후 int로 변경
                    if goangjoo[i] == '-':
                        goangjoo[i] = goangjoo[i].replace('-', str(avg_plus_avg))
                    elif goangjoo[i] == '0':
                        goangjoo[i] = goangjoo[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', goangjoo[i])
                    goangjoo[i] = int(value)

            elif lo == '대전':

                for i in range(len(daejeon)):  # 대전 가격의 문자열 제거후 int로 변경
                    if daejeon[i] == '-':
                        daejeon[i] = daejeon[i].replace('-', str(avg_plus_avg))
                    elif daejeon[i] == '0':
                        daejeon[i] = daejeon[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', daejeon[i])
                    daejeon[i] = int(value)

            # -------------------------------------------------------- 하나의 리스트에 농산물 정보 넣기

        i = 0
        z = 0

        for lo in location:

            if lo == '서울':
                for i in range(len(pro_duct1)):

                    for z in range(len(pro_duct1[0])):
                        if z == 0:
                            try:
                                pro_duct1[i][z] = seoul[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct1[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct1[i][z] = '서울'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct1[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list1[i][z] = seoul[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list1[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list1[i][z] = '서울'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list1[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '부산':
                for i in range(len(pro_duct2)):

                    for z in range(len(pro_duct2[0])):
                        if z == 0:
                            try:
                                pro_duct2[i][z] = busan[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct2[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct2[i][z] = '부산'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct2[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list2[i][z] = busan[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list2[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list2[i][z] = '부산'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list2[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '대구':

                for i in range(len(pro_duct3)):

                    for z in range(len(pro_duct3[0])):
                        if z == 0:
                            try:
                                pro_duct3[i][z] = daegoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct3[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct3[i][z] = '대구'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct3[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list3[i][z] = daegoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list3[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list3[i][z] = '대구'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list3[i][z] = lost_date[i]
                            except:
                                continue


            elif lo == '광주':

                for i in range(len(pro_duct4)):

                    for z in range(len(pro_duct4[0])):
                        if z == 0:
                            try:
                                pro_duct4[i][z] = goangjoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct4[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct4[i][z] = '광주'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct4[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list4[i][z] = goangjoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list4[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list4[i][z] = '광주'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list4[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '대전':

                for i in range(len(pro_duct5)):

                    for z in range(len(pro_duct5[0])):
                        if z == 0:
                            try:
                                pro_duct5[i][z] = daejeon[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct5[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct5[i][z] = '대전'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct5[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list5[i][z] = daejeon[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list5[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list5[i][z] = '대전'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list5[i][z] = lost_date[i]
                            except:
                                continue

        sum_pro_data1 = pro_duct1 + sum_fake_list1
        sum_pro_data2 = pro_duct2 + sum_fake_list2
        sum_pro_data3 = pro_duct3 + sum_fake_list3
        sum_pro_data4 = pro_duct4 + sum_fake_list4
        sum_pro_data5 = pro_duct5 + sum_fake_list5
        loca_data1 = sum_pro_data1 + sum_pro_data2 + sum_pro_data3 + sum_pro_data4 + sum_pro_data5



    elif kindcode[pro] == '콩':
        print(kindcode[pro], '  데이터 정제 시작')

        df_list = df.values.tolist()
        df_col = list([col for col in df])
        del df_col[0]

        pro_date = [0 for i in range(len(df_list[0]) - 1)]
        avg = [0 for i in range(len(df_list[0]) - 1)]  # 전체 평균값
        avg_fake = [0 for i in range(len(df_list[0]) - 2)]
        seoul = [0 for i in range(len(df_list[0]) - 1)]  # 서울 평균값
        busan = [0 for i in range(len(df_list[0]) - 1)]  # 부산 평균값
        daegoo = [0 for i in range(len(df_list[0]) - 1)]  # 대구 평균값
        goangjoo = [0 for i in range(len(df_list[0]) - 1)]  # 광주 평균값
        daejeon = [0 for i in range(len(df_list[0]) - 1)]  # 대전 평균값

        # --------------------------------------------------------데이터상의 날짜 구하기
        for i in range(len(df_col)):
            pp = word_tokenize(str(df_col[i]))
            z = pp[0]
            try:

                pro_date[i] = z
            except:
                continue
        i = 0

        for k in range(len(pro_date)):
            pro_date[k] = pro_date[k].replace('2021', '2020')

        # --------------------------------------------------------

        # ========================================================평균가격,서울,부산,대구,광주,대전 별로 리스트 분리

        for i in range(len(avg)):
            if i == 0:
                try:
                    avg = df_list[i][1:-1]
                except:
                    continue
            elif i == 1:
                continue

            elif i == 2:
                try:
                    seoul = df_list[i][1:]
                except:
                    continue
            elif i == 3:
                try:
                    busan = df_list[i][1:]
                except:
                    continue

            elif i == 4:
                try:
                    daegoo = df_list[i][1:]
                except:
                    continue
            elif i == 5:
                try:
                    goangjoo = df_list[i][1:]
                except:
                    continue
            elif i == 6:
                try:
                    daejeon = df_list[i][1:]
                except:
                    continue

        # ========================================================

        location = {'서울': 'seoul', '부산': 'busan', '대구': 'daegoo', '광주': 'goangjoo', '대전': 'daejeon'}
        pro_duct1 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct2 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct3 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct4 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct5 = [[0 for col in range(4)] for row in range(len(pro_date))]

        fakelist = [0 for row in range(len(datelist))]
        for i in range(len(pro_date)):
            fakelist[i] = pro_date[i]

        lost_date = list(set(datelist) - set(fakelist))

        sum_fake_list1 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list2 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list3 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list4 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list5 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        i = 0

        # -------------------------------------------------------------------평균가의 평균가격을 구함
        for i in range(len(avg)):
            value = avg[i].replace(',', '')
            avg[i] = int(value)

        for fake in range(len(avg_fake)):
            try:
                avg_fake[fake] = avg[fake]
            except:
                continue

        i = 0

        for i in range(len(avg_fake)):
            try:
                avg_fake[i] += avg_fake[i - 1]
                if i == (len(avg) - 1):
                    avg_fake[i] = avg_fake[i] - avg[i]
            except:
                continue

        avg_plus_avg = int((avg_fake[(len(avg_fake) - 1)]) / len(avg_fake))

        for lo in location:

            i = 0

            if lo == '서울':

                for i in range(len(seoul)):  # 서울 가격의 문자열 제거후 int로 변경
                    if seoul[i] == '-':
                        seoul[i] = seoul[i].replace('-', str(avg_plus_avg))
                    elif seoul[i] == '0':
                        seoul[i] = seoul[i].replace('0', str(avg_plus_avg))

                    value = re.sub(',', '', seoul[i])
                    seoul[i] = int(value)



            elif lo == '부산':

                for i in range(len(busan)):  # 부산 가격의 문자열 제거후 int로 변경
                    if busan[i] == '-':
                        busan[i] = busan[i].replace('-', str(avg_plus_avg))
                    elif busan[i] == '0':
                        busan[i] = busan[i].replace('0', str(avg_plus_avg))

                    value = re.sub(',', '', busan[i])
                    busan[i] = int(value)

            elif lo == '대구':

                for i in range(len(daegoo)):  # 대구 가격의 문자열 제거후 int로 변경
                    if daegoo[i] == '-':
                        daegoo[i] = daegoo[i].replace('-', str(avg_plus_avg))
                    elif daegoo[i] == '0':
                        daegoo[i] = daegoo[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', daegoo[i])
                    daegoo[i] = int(value)

            elif lo == '광주':

                for i in range(len(goangjoo)):  # 광주 가격의 문자열 제거후 int로 변경
                    if goangjoo[i] == '-':
                        goangjoo[i] = goangjoo[i].replace('-', str(avg_plus_avg))
                    elif goangjoo[i] == '0':
                        goangjoo[i] = goangjoo[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', goangjoo[i])
                    goangjoo[i] = int(value)

            elif lo == '대전':

                for i in range(len(daejeon)):  # 대전 가격의 문자열 제거후 int로 변경
                    if daejeon[i] == '-':
                        daejeon[i] = daejeon[i].replace('-', str(avg_plus_avg))
                    elif daejeon[i] == '0':
                        daejeon[i] = daejeon[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', daejeon[i])
                    daejeon[i] = int(value)

            # -------------------------------------------------------- 하나의 리스트에 농산물 정보 넣기

        i = 0
        z = 0

        for lo in location:

            if lo == '서울':
                for i in range(len(pro_duct1)):

                    for z in range(len(pro_duct1[0])):
                        if z == 0:
                            try:
                                pro_duct1[i][z] = seoul[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct1[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct1[i][z] = '서울'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct1[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list1[i][z] = seoul[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list1[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list1[i][z] = '서울'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list1[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '부산':
                for i in range(len(pro_duct2)):

                    for z in range(len(pro_duct2[0])):
                        if z == 0:
                            try:
                                pro_duct2[i][z] = busan[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct2[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct2[i][z] = '부산'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct2[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list2[i][z] = busan[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list2[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list2[i][z] = '부산'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list2[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '대구':

                for i in range(len(pro_duct3)):

                    for z in range(len(pro_duct3[0])):
                        if z == 0:
                            try:
                                pro_duct3[i][z] = daegoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct3[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct3[i][z] = '대구'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct3[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list3[i][z] = daegoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list3[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list3[i][z] = '대구'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list3[i][z] = lost_date[i]
                            except:
                                continue


            elif lo == '광주':

                for i in range(len(pro_duct4)):

                    for z in range(len(pro_duct4[0])):
                        if z == 0:
                            try:
                                pro_duct4[i][z] = goangjoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct4[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct4[i][z] = '광주'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct4[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list4[i][z] = goangjoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list4[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list4[i][z] = '광주'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list4[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '대전':

                for i in range(len(pro_duct5)):

                    for z in range(len(pro_duct5[0])):
                        if z == 0:
                            try:
                                pro_duct5[i][z] = daejeon[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct5[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct5[i][z] = '대전'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct5[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list5[i][z] = daejeon[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list5[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list5[i][z] = '대전'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list5[i][z] = lost_date[i]
                            except:
                                continue

        sum_pro_data1 = pro_duct1 + sum_fake_list1
        sum_pro_data2 = pro_duct2 + sum_fake_list2
        sum_pro_data3 = pro_duct3 + sum_fake_list3
        sum_pro_data4 = pro_duct4 + sum_fake_list4
        sum_pro_data5 = pro_duct5 + sum_fake_list5
        loca_data2 = sum_pro_data1 + sum_pro_data2 + sum_pro_data3 + sum_pro_data4 + sum_pro_data5


    elif kindcode[pro] == '팥':
        print(kindcode[pro], '  데이터 정제 시작')

        df_list = df.values.tolist()
        df_col = list([col for col in df])
        del df_col[0]

        pro_date = [0 for i in range(len(df_list[0]) - 1)]
        avg = [0 for i in range(len(df_list[0]) - 1)]  # 전체 평균값
        avg_fake = [0 for i in range(len(df_list[0]) - 2)]
        seoul = [0 for i in range(len(df_list[0]) - 1)]  # 서울 평균값
        busan = [0 for i in range(len(df_list[0]) - 1)]  # 부산 평균값
        daegoo = [0 for i in range(len(df_list[0]) - 1)]  # 대구 평균값
        goangjoo = [0 for i in range(len(df_list[0]) - 1)]  # 광주 평균값
        daejeon = [0 for i in range(len(df_list[0]) - 1)]  # 대전 평균값

        # --------------------------------------------------------데이터상의 날짜 구하기
        for i in range(len(df_col)):
            pp = word_tokenize(str(df_col[i]))
            z = pp[0]
            try:

                pro_date[i] = z
            except:
                continue
        i = 0

        for k in range(len(pro_date)):
            pro_date[k] = pro_date[k].replace('2021', '2020')

        # --------------------------------------------------------

        # ========================================================평균가격,서울,부산,대구,광주,대전 별로 리스트 분리

        for i in range(len(avg)):
            if i == 0:
                try:
                    avg = df_list[i][1:-1]
                except:
                    continue
            elif i == 1:
                continue

            elif i == 2:
                try:
                    seoul = df_list[i][1:]
                except:
                    continue
            elif i == 3:
                try:
                    busan = df_list[i][1:]
                except:
                    continue

            elif i == 4:
                try:
                    daegoo = df_list[i][1:]
                except:
                    continue
            elif i == 5:
                try:
                    goangjoo = df_list[i][1:]
                except:
                    continue
            elif i == 6:
                try:
                    daejeon = df_list[i][1:]
                except:
                    continue

        # ========================================================

        location = {'서울': 'seoul', '부산': 'busan', '대구': 'daegoo', '광주': 'goangjoo', '대전': 'daejeon'}
        pro_duct1 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct2 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct3 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct4 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct5 = [[0 for col in range(4)] for row in range(len(pro_date))]

        fakelist = [0 for row in range(len(datelist))]
        for i in range(len(pro_date)):
            fakelist[i] = pro_date[i]

        lost_date = list(set(datelist) - set(fakelist))

        sum_fake_list1 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list2 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list3 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list4 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list5 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        i = 0

        # -------------------------------------------------------------------평균가의 평균가격을 구함
        for i in range(len(avg)):
            value = avg[i].replace(',', '')
            avg[i] = int(value)

        for fake in range(len(avg_fake)):
            try:
                avg_fake[fake] = avg[fake]
            except:
                continue

        i = 0

        for i in range(len(avg_fake)):
            try:
                avg_fake[i] += avg_fake[i - 1]
                if i == (len(avg) - 1):
                    avg_fake[i] = avg_fake[i] - avg[i]
            except:
                continue

        avg_plus_avg = int((avg_fake[(len(avg_fake) - 1)]) / len(avg_fake))

        for lo in location:

            i = 0

            if lo == '서울':

                for i in range(len(seoul)):  # 서울 가격의 문자열 제거후 int로 변경
                    if seoul[i] == '-':
                        seoul[i] = seoul[i].replace('-', str(avg_plus_avg))
                    elif seoul[i] == '0':
                        seoul[i] = seoul[i].replace('0', str(avg_plus_avg))

                    value = re.sub(',', '', seoul[i])
                    seoul[i] = int(value)



            elif lo == '부산':

                for i in range(len(busan)):  # 부산 가격의 문자열 제거후 int로 변경
                    if busan[i] == '-':
                        busan[i] = busan[i].replace('-', str(avg_plus_avg))
                    elif busan[i] == '0':
                        busan[i] = busan[i].replace('0', str(avg_plus_avg))

                    value = re.sub(',', '', busan[i])
                    busan[i] = int(value)

            elif lo == '대구':

                for i in range(len(daegoo)):  # 대구 가격의 문자열 제거후 int로 변경
                    if daegoo[i] == '-':
                        daegoo[i] = daegoo[i].replace('-', str(avg_plus_avg))
                    elif daegoo[i] == '0':
                        daegoo[i] = daegoo[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', daegoo[i])
                    daegoo[i] = int(value)

            elif lo == '광주':

                for i in range(len(goangjoo)):  # 광주 가격의 문자열 제거후 int로 변경
                    if goangjoo[i] == '-':
                        goangjoo[i] = goangjoo[i].replace('-', str(avg_plus_avg))
                    elif goangjoo[i] == '0':
                        goangjoo[i] = goangjoo[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', goangjoo[i])
                    goangjoo[i] = int(value)

            elif lo == '대전':

                for i in range(len(daejeon)):  # 대전 가격의 문자열 제거후 int로 변경
                    if daejeon[i] == '-':
                        daejeon[i] = daejeon[i].replace('-', str(avg_plus_avg))
                    elif daejeon[i] == '0':
                        daejeon[i] = daejeon[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', daejeon[i])
                    daejeon[i] = int(value)

            # -------------------------------------------------------- 하나의 리스트에 농산물 정보 넣기

        i = 0
        z = 0

        for lo in location:

            if lo == '서울':
                for i in range(len(pro_duct1)):

                    for z in range(len(pro_duct1[0])):
                        if z == 0:
                            try:
                                pro_duct1[i][z] = seoul[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct1[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct1[i][z] = '서울'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct1[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list1[i][z] = seoul[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list1[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list1[i][z] = '서울'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list1[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '부산':
                for i in range(len(pro_duct2)):

                    for z in range(len(pro_duct2[0])):
                        if z == 0:
                            try:
                                pro_duct2[i][z] = busan[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct2[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct2[i][z] = '부산'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct2[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list2[i][z] = busan[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list2[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list2[i][z] = '부산'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list2[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '대구':

                for i in range(len(pro_duct3)):

                    for z in range(len(pro_duct3[0])):
                        if z == 0:
                            try:
                                pro_duct3[i][z] = daegoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct3[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct3[i][z] = '대구'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct3[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list3[i][z] = daegoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list3[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list3[i][z] = '대구'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list3[i][z] = lost_date[i]
                            except:
                                continue


            elif lo == '광주':

                for i in range(len(pro_duct4)):

                    for z in range(len(pro_duct4[0])):
                        if z == 0:
                            try:
                                pro_duct4[i][z] = goangjoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct4[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct4[i][z] = '광주'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct4[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list4[i][z] = goangjoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list4[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list4[i][z] = '광주'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list4[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '대전':

                for i in range(len(pro_duct5)):

                    for z in range(len(pro_duct5[0])):
                        if z == 0:
                            try:
                                pro_duct5[i][z] = daejeon[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct5[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct5[i][z] = '대전'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct5[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list5[i][z] = daejeon[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list5[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list5[i][z] = '대전'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list5[i][z] = lost_date[i]
                            except:
                                continue

        sum_pro_data1 = pro_duct1 + sum_fake_list1
        sum_pro_data2 = pro_duct2 + sum_fake_list2
        sum_pro_data3 = pro_duct3 + sum_fake_list3
        sum_pro_data4 = pro_duct4 + sum_fake_list4
        sum_pro_data5 = pro_duct5 + sum_fake_list5
        loca_data3 = sum_pro_data1 + sum_pro_data2 + sum_pro_data3 + sum_pro_data4 + sum_pro_data5

    elif kindcode[pro] == '고구마':
        print(kindcode[pro], '  데이터 정제 시작')

        df_list = df.values.tolist()
        df_col = list([col for col in df])
        del df_col[0]

        pro_date = [0 for i in range(len(df_list[0]) - 1)]
        avg = [0 for i in range(len(df_list[0]) - 1)]  # 전체 평균값
        avg_fake = [0 for i in range(len(df_list[0]) - 2)]
        seoul = [0 for i in range(len(df_list[0]) - 1)]  # 서울 평균값
        busan = [0 for i in range(len(df_list[0]) - 1)]  # 부산 평균값
        daegoo = [0 for i in range(len(df_list[0]) - 1)]  # 대구 평균값
        goangjoo = [0 for i in range(len(df_list[0]) - 1)]  # 광주 평균값
        daejeon = [0 for i in range(len(df_list[0]) - 1)]  # 대전 평균값

        # --------------------------------------------------------데이터상의 날짜 구하기
        for i in range(len(df_col)):
            pp = word_tokenize(str(df_col[i]))
            z = pp[0]
            try:

                pro_date[i] = z
            except:
                continue
        i = 0

        for k in range(len(pro_date)):
            pro_date[k] = pro_date[k].replace('2021', '2020')

        # --------------------------------------------------------

        # ========================================================평균가격,서울,부산,대구,광주,대전 별로 리스트 분리

        for i in range(len(avg)):
            if i == 0:
                try:
                    avg = df_list[i][1:-1]
                except:
                    continue
            elif i == 1:
                continue

            elif i == 2:
                try:
                    seoul = df_list[i][1:]
                except:
                    continue
            elif i == 3:
                try:
                    busan = df_list[i][1:]
                except:
                    continue

            elif i == 4:
                try:
                    daegoo = df_list[i][1:]
                except:
                    continue
            elif i == 5:
                try:
                    goangjoo = df_list[i][1:]
                except:
                    continue
            elif i == 6:
                try:
                    daejeon = df_list[i][1:]
                except:
                    continue

        # ========================================================

        location = {'서울': 'seoul', '부산': 'busan', '대구': 'daegoo', '광주': 'goangjoo', '대전': 'daejeon'}
        pro_duct1 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct2 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct3 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct4 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct5 = [[0 for col in range(4)] for row in range(len(pro_date))]

        fakelist = [0 for row in range(len(datelist))]
        for i in range(len(pro_date)):
            fakelist[i] = pro_date[i]

        lost_date = list(set(datelist) - set(fakelist))

        sum_fake_list1 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list2 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list3 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list4 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list5 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        i = 0

        # -------------------------------------------------------------------평균가의 평균가격을 구함
        for i in range(len(avg)):
            value = avg[i].replace(',', '')

            avg[i] = int(value)

        for fake in range(len(avg_fake)):
            try:
                avg_fake[fake] = avg[fake]
            except:
                continue

        i = 0

        for i in range(len(avg_fake)):
            try:
                avg_fake[i] += avg_fake[i - 1]
                if i == (len(avg) - 1):
                    avg_fake[i] = avg_fake[i] - avg[i]
            except:
                continue

        avg_plus_avg = int((avg_fake[(len(avg_fake) - 1)]) / len(avg_fake))

        for lo in location:

            i = 0

            if lo == '서울':

                for i in range(len(seoul)):  # 서울 가격의 문자열 제거후 int로 변경
                    if seoul[i] == '-':
                        seoul[i] = seoul[i].replace('-', str(avg_plus_avg))
                    elif seoul[i] == '0':
                        seoul[i] = seoul[i].replace('0', str(avg_plus_avg))

                    value = re.sub(',', '', seoul[i])
                    seoul[i] = int(value)



            elif lo == '부산':

                for i in range(len(busan)):  # 부산 가격의 문자열 제거후 int로 변경
                    if busan[i] == '-':
                        busan[i] = busan[i].replace('-', str(avg_plus_avg))
                    elif busan[i] == '0':
                        busan[i] = busan[i].replace('0', str(avg_plus_avg))

                    value = re.sub(',', '', busan[i])
                    busan[i] = int(value)

            elif lo == '대구':

                for i in range(len(daegoo)):  # 대구 가격의 문자열 제거후 int로 변경
                    if daegoo[i] == '-':
                        daegoo[i] = daegoo[i].replace('-', str(avg_plus_avg))
                    elif daegoo[i] == '0':
                        daegoo[i] = daegoo[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', daegoo[i])
                    daegoo[i] = int(value)

            elif lo == '광주':

                for i in range(len(goangjoo)):  # 광주 가격의 문자열 제거후 int로 변경
                    if goangjoo[i] == '-':
                        goangjoo[i] = goangjoo[i].replace('-', str(avg_plus_avg))
                    elif goangjoo[i] == '0':
                        goangjoo[i] = goangjoo[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', goangjoo[i])
                    goangjoo[i] = int(value)

            elif lo == '대전':

                for i in range(len(daejeon)):  # 대전 가격의 문자열 제거후 int로 변경
                    if daejeon[i] == '-':
                        daejeon[i] = daejeon[i].replace('-', str(avg_plus_avg))
                    elif daejeon[i] == '0':
                        daejeon[i] = daejeon[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', daejeon[i])
                    daejeon[i] = int(value)

            # -------------------------------------------------------- 하나의 리스트에 농산물 정보 넣기

        i = 0
        z = 0

        for lo in location:

            if lo == '서울':
                for i in range(len(pro_duct1)):

                    for z in range(len(pro_duct1[0])):
                        if z == 0:
                            try:
                                pro_duct1[i][z] = seoul[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct1[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct1[i][z] = '서울'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct1[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list1[i][z] = seoul[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list1[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list1[i][z] = '서울'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list1[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '부산':
                for i in range(len(pro_duct2)):

                    for z in range(len(pro_duct2[0])):
                        if z == 0:
                            try:
                                pro_duct2[i][z] = busan[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct2[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct2[i][z] = '부산'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct2[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list2[i][z] = busan[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list2[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list2[i][z] = '부산'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list2[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '대구':

                for i in range(len(pro_duct3)):

                    for z in range(len(pro_duct3[0])):
                        if z == 0:
                            try:
                                pro_duct3[i][z] = daegoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct3[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct3[i][z] = '대구'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct3[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list3[i][z] = daegoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list3[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list3[i][z] = '대구'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list3[i][z] = lost_date[i]
                            except:
                                continue


            elif lo == '광주':

                for i in range(len(pro_duct4)):

                    for z in range(len(pro_duct4[0])):
                        if z == 0:
                            try:
                                pro_duct4[i][z] = goangjoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct4[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct4[i][z] = '광주'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct4[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list4[i][z] = goangjoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list4[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list4[i][z] = '광주'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list4[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '대전':

                for i in range(len(pro_duct5)):

                    for z in range(len(pro_duct5[0])):
                        if z == 0:
                            try:
                                pro_duct5[i][z] = daejeon[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct5[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct5[i][z] = '대전'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct5[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list5[i][z] = daejeon[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list5[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list5[i][z] = '대전'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list5[i][z] = lost_date[i]
                            except:
                                continue

        sum_pro_data1 = pro_duct1 + sum_fake_list1
        sum_pro_data2 = pro_duct2 + sum_fake_list2
        sum_pro_data3 = pro_duct3 + sum_fake_list3
        sum_pro_data4 = pro_duct4 + sum_fake_list4
        sum_pro_data5 = pro_duct5 + sum_fake_list5
        loca_data4 = sum_pro_data1 + sum_pro_data2 + sum_pro_data3 + sum_pro_data4 + sum_pro_data5



    elif kindcode[pro] == '감자':
        print(kindcode[pro], '  데이터 정제 시작')

        df_list = df.values.tolist()
        df_col = list([col for col in df])
        del df_col[0]

        pro_date = [0 for i in range(len(df_list[0]) - 1)]
        avg = [0 for i in range(len(df_list[0]) - 1)]  # 전체 평균값
        avg_fake = [0 for i in range(len(df_list[0]) - 2)]
        seoul = [0 for i in range(len(df_list[0]) - 1)]  # 서울 평균값
        busan = [0 for i in range(len(df_list[0]) - 1)]  # 부산 평균값
        daegoo = [0 for i in range(len(df_list[0]) - 1)]  # 대구 평균값
        goangjoo = [0 for i in range(len(df_list[0]) - 1)]  # 광주 평균값
        daejeon = [0 for i in range(len(df_list[0]) - 1)]  # 대전 평균값

        # --------------------------------------------------------데이터상의 날짜 구하기
        for i in range(len(df_col)):
            pp = word_tokenize(str(df_col[i]))
            z = pp[0]
            try:

                pro_date[i] = z
            except:
                continue
        i = 0

        for k in range(len(pro_date)):
            pro_date[k] = pro_date[k].replace('2021', '2020')

        # --------------------------------------------------------

        # ========================================================평균가격,서울,부산,대구,광주,대전 별로 리스트 분리

        for i in range(len(avg)):
            if i == 0:
                try:
                    avg = df_list[i][1:-1]
                except:
                    continue
            elif i == 1:
                continue

            elif i == 2:
                try:
                    seoul = df_list[i][1:]
                except:
                    continue
            elif i == 3:
                try:
                    busan = df_list[i][1:]
                except:
                    continue

            elif i == 4:
                try:
                    daegoo = df_list[i][1:]
                except:
                    continue
            elif i == 5:
                try:
                    goangjoo = df_list[i][1:]
                except:
                    continue
            elif i == 6:
                try:
                    daejeon = df_list[i][1:]
                except:
                    continue

        # ========================================================

        location = {'서울': 'seoul', '부산': 'busan', '대구': 'daegoo', '광주': 'goangjoo', '대전': 'daejeon'}
        pro_duct1 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct2 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct3 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct4 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct5 = [[0 for col in range(4)] for row in range(len(pro_date))]

        fakelist = [0 for row in range(len(datelist))]
        for i in range(len(pro_date)):
            fakelist[i] = pro_date[i]

        lost_date = list(set(datelist) - set(fakelist))

        sum_fake_list1 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list2 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list3 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list4 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list5 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        i = 0

        # -------------------------------------------------------------------평균가의 평균가격을 구함
        for i in range(len(avg)):
            value = avg[i].replace(',', '')
            avg[i] = int(value)

        for fake in range(len(avg_fake)):
            try:
                avg_fake[fake] = avg[fake]
            except:
                continue

        i = 0

        for i in range(len(avg_fake)):
            try:
                avg_fake[i] += avg_fake[i - 1]
                if i == (len(avg) - 1):
                    avg_fake[i] = avg_fake[i] - avg[i]
            except:
                continue

        avg_plus_avg = int((avg_fake[(len(avg_fake) - 1)]) / len(avg_fake))

        for lo in location:

            i = 0

            if lo == '서울':

                for i in range(len(seoul)):  # 서울 가격의 문자열 제거후 int로 변경
                    if seoul[i] == '-':
                        seoul[i] = seoul[i].replace('-', str(avg_plus_avg))
                    elif seoul[i] == '0':
                        seoul[i] = seoul[i].replace('0', str(avg_plus_avg))

                    value = re.sub(',', '', seoul[i])
                    seoul[i] = int(value)



            elif lo == '부산':

                for i in range(len(busan)):  # 부산 가격의 문자열 제거후 int로 변경
                    if busan[i] == '-':
                        busan[i] = busan[i].replace('-', str(avg_plus_avg))
                    elif busan[i] == '0':
                        busan[i] = busan[i].replace('0', str(avg_plus_avg))

                    value = re.sub(',', '', busan[i])
                    busan[i] = int(value)

            elif lo == '대구':

                for i in range(len(daegoo)):  # 대구 가격의 문자열 제거후 int로 변경
                    if daegoo[i] == '-':
                        daegoo[i] = daegoo[i].replace('-', str(avg_plus_avg))
                    elif daegoo[i] == '0':
                        daegoo[i] = daegoo[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', daegoo[i])
                    daegoo[i] = int(value)

            elif lo == '광주':

                for i in range(len(goangjoo)):  # 광주 가격의 문자열 제거후 int로 변경
                    if goangjoo[i] == '-':
                        goangjoo[i] = goangjoo[i].replace('-', str(avg_plus_avg))
                    elif goangjoo[i] == '0':
                        goangjoo[i] = goangjoo[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', goangjoo[i])
                    goangjoo[i] = int(value)

            elif lo == '대전':

                for i in range(len(daejeon)):  # 대전 가격의 문자열 제거후 int로 변경
                    if daejeon[i] == '-':
                        daejeon[i] = daejeon[i].replace('-', str(avg_plus_avg))
                    elif daejeon[i] == '0':
                        daejeon[i] = daejeon[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', daejeon[i])
                    daejeon[i] = int(value)

            # -------------------------------------------------------- 하나의 리스트에 농산물 정보 넣기

        i = 0
        z = 0

        for lo in location:

            if lo == '서울':
                for i in range(len(pro_duct1)):

                    for z in range(len(pro_duct1[0])):
                        if z == 0:
                            try:
                                pro_duct1[i][z] = seoul[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct1[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct1[i][z] = '서울'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct1[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list1[i][z] = seoul[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list1[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list1[i][z] = '서울'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list1[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '부산':
                for i in range(len(pro_duct2)):

                    for z in range(len(pro_duct2[0])):
                        if z == 0:
                            try:
                                pro_duct2[i][z] = busan[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct2[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct2[i][z] = '부산'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct2[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list2[i][z] = busan[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list2[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list2[i][z] = '부산'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list2[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '대구':

                for i in range(len(pro_duct3)):

                    for z in range(len(pro_duct3[0])):
                        if z == 0:
                            try:
                                pro_duct3[i][z] = daegoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct3[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct3[i][z] = '대구'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct3[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list3[i][z] = daegoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list3[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list3[i][z] = '대구'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list3[i][z] = lost_date[i]
                            except:
                                continue


            elif lo == '광주':

                for i in range(len(pro_duct4)):

                    for z in range(len(pro_duct4[0])):
                        if z == 0:
                            try:
                                pro_duct4[i][z] = goangjoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct4[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct4[i][z] = '광주'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct4[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list4[i][z] = goangjoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list4[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list4[i][z] = '광주'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list4[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '대전':

                for i in range(len(pro_duct5)):

                    for z in range(len(pro_duct5[0])):
                        if z == 0:
                            try:
                                pro_duct5[i][z] = daejeon[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct5[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct5[i][z] = '대전'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct5[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list5[i][z] = daejeon[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list5[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list5[i][z] = '대전'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list5[i][z] = lost_date[i]
                            except:
                                continue

        sum_pro_data1 = pro_duct1 + sum_fake_list1
        sum_pro_data2 = pro_duct2 + sum_fake_list2
        sum_pro_data3 = pro_duct3 + sum_fake_list3
        sum_pro_data4 = pro_duct4 + sum_fake_list4
        sum_pro_data5 = pro_duct5 + sum_fake_list5
        loca_data5 = sum_pro_data1 + sum_pro_data2 + sum_pro_data3 + sum_pro_data4 + sum_pro_data5


    elif kindcode[pro] == '시금치':
        print(kindcode[pro], '  데이터 정제 시작')

        df_list = df.values.tolist()
        df_col = list([col for col in df])
        del df_col[0]

        pro_date = [0 for i in range(len(df_list[0]) - 1)]
        avg = [0 for i in range(len(df_list[0]) - 1)]  # 전체 평균값
        avg_fake = [0 for i in range(len(df_list[0]) - 2)]
        seoul = [0 for i in range(len(df_list[0]) - 1)]  # 서울 평균값
        busan = [0 for i in range(len(df_list[0]) - 1)]  # 부산 평균값
        daegoo = [0 for i in range(len(df_list[0]) - 1)]  # 대구 평균값
        goangjoo = [0 for i in range(len(df_list[0]) - 1)]  # 광주 평균값
        daejeon = [0 for i in range(len(df_list[0]) - 1)]  # 대전 평균값

        # --------------------------------------------------------데이터상의 날짜 구하기
        for i in range(len(df_col)):
            pp = word_tokenize(str(df_col[i]))
            z = pp[0]
            try:

                pro_date[i] = z
            except:
                continue
        i = 0

        for k in range(len(pro_date)):
            pro_date[k] = pro_date[k].replace('2021', '2020')

        # --------------------------------------------------------

        # ========================================================평균가격,서울,부산,대구,광주,대전 별로 리스트 분리

        for i in range(len(avg)):
            if i == 0:
                try:
                    avg = df_list[i][1:-1]
                except:
                    continue
            elif i == 1:
                continue

            elif i == 2:
                try:
                    seoul = df_list[i][1:]
                except:
                    continue
            elif i == 3:
                try:
                    busan = df_list[i][1:]
                except:
                    continue

            elif i == 4:
                try:
                    daegoo = df_list[i][1:]
                except:
                    continue
            elif i == 5:
                try:
                    goangjoo = df_list[i][1:]
                except:
                    continue
            elif i == 6:
                try:
                    daejeon = df_list[i][1:]
                except:
                    continue

        # ========================================================

        location = {'서울': 'seoul', '부산': 'busan', '대구': 'daegoo', '광주': 'goangjoo', '대전': 'daejeon'}
        pro_duct1 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct2 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct3 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct4 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct5 = [[0 for col in range(4)] for row in range(len(pro_date))]

        fakelist = [0 for row in range(len(datelist))]
        for i in range(len(pro_date)):
            fakelist[i] = pro_date[i]

        lost_date = list(set(datelist) - set(fakelist))

        sum_fake_list1 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list2 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list3 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list4 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list5 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        i = 0

        # -------------------------------------------------------------------평균가의 평균가격을 구함
        for i in range(len(avg)):
            value = avg[i].replace(',', '')
            avg[i] = int(value)

        for fake in range(len(avg_fake)):
            try:
                avg_fake[fake] = avg[fake]
            except:
                continue

        i = 0

        for i in range(len(avg_fake)):
            try:
                avg_fake[i] += avg_fake[i - 1]
                if i == (len(avg) - 1):
                    avg_fake[i] = avg_fake[i] - avg[i]
            except:
                continue

        avg_plus_avg = int((avg_fake[(len(avg_fake) - 1)]) / len(avg_fake))

        for lo in location:

            i = 0

            if lo == '서울':

                for i in range(len(seoul)):  # 서울 가격의 문자열 제거후 int로 변경
                    if seoul[i] == '-':
                        seoul[i] = seoul[i].replace('-', str(avg_plus_avg))
                    elif seoul[i] == '0':
                        seoul[i] = seoul[i].replace('0', str(avg_plus_avg))

                    value = re.sub(',', '', seoul[i])
                    seoul[i] = int(value)



            elif lo == '부산':

                for i in range(len(busan)):  # 부산 가격의 문자열 제거후 int로 변경
                    if busan[i] == '-':
                        busan[i] = busan[i].replace('-', str(avg_plus_avg))
                    elif busan[i] == '0':
                        busan[i] = busan[i].replace('0', str(avg_plus_avg))

                    value = re.sub(',', '', busan[i])
                    busan[i] = int(value)

            elif lo == '대구':

                for i in range(len(daegoo)):  # 대구 가격의 문자열 제거후 int로 변경
                    if daegoo[i] == '-':
                        daegoo[i] = daegoo[i].replace('-', str(avg_plus_avg))
                    elif daegoo[i] == '0':
                        daegoo[i] = daegoo[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', daegoo[i])
                    daegoo[i] = int(value)

            elif lo == '광주':

                for i in range(len(goangjoo)):  # 광주 가격의 문자열 제거후 int로 변경
                    if goangjoo[i] == '-':
                        goangjoo[i] = goangjoo[i].replace('-', str(avg_plus_avg))
                    elif goangjoo[i] == '0':
                        goangjoo[i] = goangjoo[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', goangjoo[i])
                    goangjoo[i] = int(value)

            elif lo == '대전':

                for i in range(len(daejeon)):  # 대전 가격의 문자열 제거후 int로 변경
                    if daejeon[i] == '-':
                        daejeon[i] = daejeon[i].replace('-', str(avg_plus_avg))
                    elif daejeon[i] == '0':
                        daejeon[i] = daejeon[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', daejeon[i])
                    daejeon[i] = int(value)

            # -------------------------------------------------------- 하나의 리스트에 농산물 정보 넣기

        i = 0
        z = 0

        for lo in location:

            if lo == '서울':
                for i in range(len(pro_duct1)):

                    for z in range(len(pro_duct1[0])):
                        if z == 0:
                            try:
                                pro_duct1[i][z] = seoul[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct1[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct1[i][z] = '서울'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct1[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list1[i][z] = seoul[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list1[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list1[i][z] = '서울'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list1[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '부산':
                for i in range(len(pro_duct2)):

                    for z in range(len(pro_duct2[0])):
                        if z == 0:
                            try:
                                pro_duct2[i][z] = busan[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct2[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct2[i][z] = '부산'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct2[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list2[i][z] = busan[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list2[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list2[i][z] = '부산'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list2[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '대구':

                for i in range(len(pro_duct3)):

                    for z in range(len(pro_duct3[0])):
                        if z == 0:
                            try:
                                pro_duct3[i][z] = daegoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct3[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct3[i][z] = '대구'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct3[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list3[i][z] = daegoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list3[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list3[i][z] = '대구'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list3[i][z] = lost_date[i]
                            except:
                                continue


            elif lo == '광주':

                for i in range(len(pro_duct4)):

                    for z in range(len(pro_duct4[0])):
                        if z == 0:
                            try:
                                pro_duct4[i][z] = goangjoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct4[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct4[i][z] = '광주'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct4[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list4[i][z] = goangjoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list4[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list4[i][z] = '광주'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list4[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '대전':

                for i in range(len(pro_duct5)):

                    for z in range(len(pro_duct5[0])):
                        if z == 0:
                            try:
                                pro_duct5[i][z] = daejeon[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct5[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct5[i][z] = '대전'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct5[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list5[i][z] = daejeon[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list5[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list5[i][z] = '대전'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list5[i][z] = lost_date[i]
                            except:
                                continue

        sum_pro_data1 = pro_duct1 + sum_fake_list1
        sum_pro_data2 = pro_duct2 + sum_fake_list2
        sum_pro_data3 = pro_duct3 + sum_fake_list3
        sum_pro_data4 = pro_duct4 + sum_fake_list4
        sum_pro_data5 = pro_duct5 + sum_fake_list5
        loca_data6 = sum_pro_data1 + sum_pro_data2 + sum_pro_data3 + sum_pro_data4 + sum_pro_data5


    elif kindcode[pro] == '열무':
        print(kindcode[pro], '  데이터 정제 시작')

        df_list = df.values.tolist()
        df_col = list([col for col in df])
        del df_col[0]

        pro_date = [0 for i in range(len(df_list[0]) - 1)]
        avg = [0 for i in range(len(df_list[0]) - 1)]  # 전체 평균값
        avg_fake = [0 for i in range(len(df_list[0]) - 2)]
        seoul = [0 for i in range(len(df_list[0]) - 1)]  # 서울 평균값
        busan = [0 for i in range(len(df_list[0]) - 1)]  # 부산 평균값
        daegoo = [0 for i in range(len(df_list[0]) - 1)]  # 대구 평균값
        goangjoo = [0 for i in range(len(df_list[0]) - 1)]  # 광주 평균값
        daejeon = [0 for i in range(len(df_list[0]) - 1)]  # 대전 평균값

        # --------------------------------------------------------데이터상의 날짜 구하기
        for i in range(len(df_col)):
            pp = word_tokenize(str(df_col[i]))
            z = pp[0]
            try:

                pro_date[i] = z
            except:
                continue
        i = 0

        for k in range(len(pro_date)):
            pro_date[k] = pro_date[k].replace('2021', '2020')

        # --------------------------------------------------------

        # ========================================================평균가격,서울,부산,대구,광주,대전 별로 리스트 분리

        for i in range(len(avg)):
            if i == 0:
                try:
                    avg = df_list[i][1:-1]
                except:
                    continue
            elif i == 1:
                continue

            elif i == 2:
                try:
                    seoul = df_list[i][1:]
                except:
                    continue
            elif i == 3:
                try:
                    busan = df_list[i][1:]
                except:
                    continue

            elif i == 4:
                try:
                    daegoo = df_list[i][1:]
                except:
                    continue
            elif i == 5:
                try:
                    goangjoo = df_list[i][1:]
                except:
                    continue
            elif i == 6:
                try:
                    daejeon = df_list[i][1:]
                except:
                    continue

        # ========================================================

        location = {'서울': 'seoul', '부산': 'busan', '대구': 'daegoo', '광주': 'goangjoo', '대전': 'daejeon'}
        pro_duct1 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct2 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct3 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct4 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct5 = [[0 for col in range(4)] for row in range(len(pro_date))]

        fakelist = [0 for row in range(len(datelist))]
        for i in range(len(pro_date)):
            fakelist[i] = pro_date[i]

        lost_date = list(set(datelist) - set(fakelist))

        sum_fake_list1 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list2 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list3 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list4 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list5 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        i = 0

        # -------------------------------------------------------------------평균가의 평균가격을 구함
        for i in range(len(avg)):
            value = avg[i].replace(',', '')
            avg[i] = int(value)

        for fake in range(len(avg_fake)):
            try:
                avg_fake[fake] = avg[fake]
            except:
                continue

        i = 0

        for i in range(len(avg_fake)):
            try:
                avg_fake[i] += avg_fake[i - 1]
                if i == (len(avg) - 1):
                    avg_fake[i] = avg_fake[i] - avg[i]
            except:
                continue

        avg_plus_avg = int((avg_fake[(len(avg_fake) - 1)]) / len(avg_fake))

        for lo in location:

            i = 0

            if lo == '서울':

                for i in range(len(seoul)):  # 서울 가격의 문자열 제거후 int로 변경
                    if seoul[i] == '-':
                        seoul[i] = seoul[i].replace('-', str(avg_plus_avg))
                    elif seoul[i] == '0':
                        seoul[i] = seoul[i].replace('0', str(avg_plus_avg))

                    value = re.sub(',', '', seoul[i])
                    seoul[i] = int(value)



            elif lo == '부산':

                for i in range(len(busan)):  # 부산 가격의 문자열 제거후 int로 변경
                    if busan[i] == '-':
                        busan[i] = busan[i].replace('-', str(avg_plus_avg))
                    elif busan[i] == '0':
                        busan[i] = busan[i].replace('0', str(avg_plus_avg))

                    value = re.sub(',', '', busan[i])
                    busan[i] = int(value)

            elif lo == '대구':

                for i in range(len(daegoo)):  # 대구 가격의 문자열 제거후 int로 변경
                    if daegoo[i] == '-':
                        daegoo[i] = daegoo[i].replace('-', str(avg_plus_avg))
                    elif daegoo[i] == '0':
                        daegoo[i] = daegoo[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', daegoo[i])
                    daegoo[i] = int(value)

            elif lo == '광주':

                for i in range(len(goangjoo)):  # 광주 가격의 문자열 제거후 int로 변경
                    if goangjoo[i] == '-':
                        goangjoo[i] = goangjoo[i].replace('-', str(avg_plus_avg))
                    elif goangjoo[i] == '0':
                        goangjoo[i] = goangjoo[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', goangjoo[i])
                    goangjoo[i] = int(value)

            elif lo == '대전':

                for i in range(len(daejeon)):  # 대전 가격의 문자열 제거후 int로 변경
                    if daejeon[i] == '-':
                        daejeon[i] = daejeon[i].replace('-', str(avg_plus_avg))
                    elif daejeon[i] == '0':
                        daejeon[i] = daejeon[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', daejeon[i])
                    daejeon[i] = int(value)

            # -------------------------------------------------------- 하나의 리스트에 농산물 정보 넣기

        i = 0
        z = 0

        for lo in location:

            if lo == '서울':
                for i in range(len(pro_duct1)):

                    for z in range(len(pro_duct1[0])):
                        if z == 0:
                            try:
                                pro_duct1[i][z] = seoul[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct1[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct1[i][z] = '서울'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct1[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list1[i][z] = seoul[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list1[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list1[i][z] = '서울'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list1[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '부산':
                for i in range(len(pro_duct2)):

                    for z in range(len(pro_duct2[0])):
                        if z == 0:
                            try:
                                pro_duct2[i][z] = busan[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct2[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct2[i][z] = '부산'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct2[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list2[i][z] = busan[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list2[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list2[i][z] = '부산'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list2[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '대구':

                for i in range(len(pro_duct3)):

                    for z in range(len(pro_duct3[0])):
                        if z == 0:
                            try:
                                pro_duct3[i][z] = daegoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct3[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct3[i][z] = '대구'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct3[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list3[i][z] = daegoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list3[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list3[i][z] = '대구'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list3[i][z] = lost_date[i]
                            except:
                                continue


            elif lo == '광주':

                for i in range(len(pro_duct4)):

                    for z in range(len(pro_duct4[0])):
                        if z == 0:
                            try:
                                pro_duct4[i][z] = goangjoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct4[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct4[i][z] = '광주'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct4[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list4[i][z] = goangjoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list4[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list4[i][z] = '광주'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list4[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '대전':

                for i in range(len(pro_duct5)):

                    for z in range(len(pro_duct5[0])):
                        if z == 0:
                            try:
                                pro_duct5[i][z] = daejeon[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct5[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct5[i][z] = '대전'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct5[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list5[i][z] = daejeon[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list5[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list5[i][z] = '대전'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list5[i][z] = lost_date[i]
                            except:
                                continue

        sum_pro_data1 = pro_duct1 + sum_fake_list1
        sum_pro_data2 = pro_duct2 + sum_fake_list2
        sum_pro_data3 = pro_duct3 + sum_fake_list3
        sum_pro_data4 = pro_duct4 + sum_fake_list4
        sum_pro_data5 = pro_duct5 + sum_fake_list5
        loca_data7 = sum_pro_data1 + sum_pro_data2 + sum_pro_data3 + sum_pro_data4 + sum_pro_data5

    elif kindcode[pro] == '대파':
        print(kindcode[pro], '  데이터 정제 시작')

        df_list = df.values.tolist()
        df_col = list([col for col in df])
        del df_col[0]

        pro_date = [0 for i in range(len(df_list[0]) - 1)]
        avg = [0 for i in range(len(df_list[0]) - 1)]  # 전체 평균값
        avg_fake = [0 for i in range(len(df_list[0]) - 2)]
        seoul = [0 for i in range(len(df_list[0]) - 1)]  # 서울 평균값
        busan = [0 for i in range(len(df_list[0]) - 1)]  # 부산 평균값
        daegoo = [0 for i in range(len(df_list[0]) - 1)]  # 대구 평균값
        goangjoo = [0 for i in range(len(df_list[0]) - 1)]  # 광주 평균값
        daejeon = [0 for i in range(len(df_list[0]) - 1)]  # 대전 평균값

        # --------------------------------------------------------데이터상의 날짜 구하기
        for i in range(len(df_col)):
            pp = word_tokenize(str(df_col[i]))
            z = pp[0]
            try:

                pro_date[i] = z
            except:
                continue
        i = 0

        for k in range(len(pro_date)):
            pro_date[k] = pro_date[k].replace('2021', '2020')

        # --------------------------------------------------------

        # ========================================================평균가격,서울,부산,대구,광주,대전 별로 리스트 분리

        for i in range(len(avg)):
            if i == 0:
                try:
                    avg = df_list[i][1:-1]
                except:
                    continue
            elif i == 1:
                continue

            elif i == 2:
                try:
                    seoul = df_list[i][1:]
                except:
                    continue
            elif i == 3:
                try:
                    busan = df_list[i][1:]
                except:
                    continue

            elif i == 4:
                try:
                    daegoo = df_list[i][1:]
                except:
                    continue
            elif i == 5:
                try:
                    goangjoo = df_list[i][1:]
                except:
                    continue
            elif i == 6:
                try:
                    daejeon = df_list[i][1:]
                except:
                    continue

        # ========================================================

        location = {'서울': 'seoul', '부산': 'busan', '대구': 'daegoo', '광주': 'goangjoo', '대전': 'daejeon'}
        pro_duct1 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct2 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct3 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct4 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct5 = [[0 for col in range(4)] for row in range(len(pro_date))]

        fakelist = [0 for row in range(len(datelist))]
        for i in range(len(pro_date)):
            fakelist[i] = pro_date[i]

        lost_date = list(set(datelist) - set(fakelist))

        sum_fake_list1 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list2 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list3 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list4 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list5 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        i = 0

        # -------------------------------------------------------------------평균가의 평균가격을 구함
        for i in range(len(avg)):
            value = avg[i].replace(',', '')
            avg[i] = int(value)

        for fake in range(len(avg_fake)):
            try:
                avg_fake[fake] = avg[fake]
            except:
                continue

        i = 0

        for i in range(len(avg_fake)):
            try:
                avg_fake[i] += avg_fake[i - 1]
                if i == (len(avg) - 1):
                    avg_fake[i] = avg_fake[i] - avg[i]
            except:
                continue

        avg_plus_avg = int((avg_fake[(len(avg_fake) - 1)]) / len(avg_fake))

        for lo in location:

            i = 0

            if lo == '서울':

                for i in range(len(seoul)):  # 서울 가격의 문자열 제거후 int로 변경
                    if seoul[i] == '-':
                        seoul[i] = seoul[i].replace('-', str(avg_plus_avg))
                    elif seoul[i] == '0':
                        seoul[i] = seoul[i].replace('0', str(avg_plus_avg))

                    value = re.sub(',', '', seoul[i])
                    seoul[i] = int(value)



            elif lo == '부산':

                for i in range(len(busan)):  # 부산 가격의 문자열 제거후 int로 변경
                    if busan[i] == '-' or busan[i] == '0':
                        busan[i] = busan[i].replace('-', str(avg_plus_avg))
                    elif busan[i] == '0':
                        busan[i] = busan[i].replace('0', str(avg_plus_avg))

                    value = re.sub(',', '', busan[i])
                    busan[i] = int(value)

            elif lo == '대구':

                for i in range(len(daegoo)):  # 대구 가격의 문자열 제거후 int로 변경
                    if daegoo[i] == '-' or daegoo[i] == '0':
                        daegoo[i] = daegoo[i].replace('-', str(avg_plus_avg))
                    elif daegoo[i] == '0':
                        daegoo[i] = daegoo[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', daegoo[i])
                    daegoo[i] = int(value)

            elif lo == '광주':

                for i in range(len(goangjoo)):  # 광주 가격의 문자열 제거후 int로 변경
                    if goangjoo[i] == '-' or goangjoo[i] == '0':
                        goangjoo[i] = goangjoo[i].replace('-', str(avg_plus_avg))
                    elif goangjoo[i] == '0':
                        goangjoo[i] = goangjoo[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', goangjoo[i])
                    goangjoo[i] = int(value)

            elif lo == '대전':

                for i in range(len(daejeon)):  # 대전 가격의 문자열 제거후 int로 변경
                    if daejeon[i] == '-' or daejeon[i] == '0':
                        daejeon[i] = daejeon[i].replace('-', str(avg_plus_avg))
                    elif daejeon[i] == '0':
                        daejeon[i] = daejeon[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', daejeon[i])
                    daejeon[i] = int(value)

            # -------------------------------------------------------- 하나의 리스트에 농산물 정보 넣기

        i = 0
        z = 0

        for lo in location:

            if lo == '서울':
                for i in range(len(pro_duct1)):

                    for z in range(len(pro_duct1[0])):
                        if z == 0:
                            try:
                                pro_duct1[i][z] = seoul[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct1[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct1[i][z] = '서울'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct1[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list1[i][z] = seoul[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list1[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list1[i][z] = '서울'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list1[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '부산':
                for i in range(len(pro_duct2)):

                    for z in range(len(pro_duct2[0])):
                        if z == 0:
                            try:
                                pro_duct2[i][z] = busan[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct2[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct2[i][z] = '부산'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct2[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list2[i][z] = busan[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list2[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list2[i][z] = '부산'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list2[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '대구':

                for i in range(len(pro_duct3)):

                    for z in range(len(pro_duct3[0])):
                        if z == 0:
                            try:
                                pro_duct3[i][z] = daegoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct3[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct3[i][z] = '대구'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct3[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list3[i][z] = daegoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list3[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list3[i][z] = '대구'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list3[i][z] = lost_date[i]
                            except:
                                continue


            elif lo == '광주':

                for i in range(len(pro_duct4)):

                    for z in range(len(pro_duct4[0])):
                        if z == 0:
                            try:
                                pro_duct4[i][z] = goangjoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct4[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct4[i][z] = '광주'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct4[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list4[i][z] = goangjoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list4[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list4[i][z] = '광주'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list4[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '대전':

                for i in range(len(pro_duct5)):

                    for z in range(len(pro_duct5[0])):
                        if z == 0:
                            try:
                                pro_duct5[i][z] = daejeon[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct5[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct5[i][z] = '대전'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct5[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list5[i][z] = daejeon[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list5[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list5[i][z] = '대전'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list5[i][z] = lost_date[i]
                            except:
                                continue

        sum_pro_data1 = pro_duct1 + sum_fake_list1
        sum_pro_data2 = pro_duct2 + sum_fake_list2
        sum_pro_data3 = pro_duct3 + sum_fake_list3
        sum_pro_data4 = pro_duct4 + sum_fake_list4
        sum_pro_data5 = pro_duct5 + sum_fake_list5
        loca_data8 = sum_pro_data1 + sum_pro_data2 + sum_pro_data3 + sum_pro_data4 + sum_pro_data5

###################################################################################################################################
# 날짜를 비교하기 위한 기상데이터

f2 = open(year + '대전기온.csv')  # 이전년도 기온.csv
temperature_data1 = csv.reader(f2)
header = next(temperature_data1)
cnt = 0
temperature_realdata1 = []
for row in temperature_data1:
    cnt += 1
    if (cnt > 11):
        temperature_realdata1.append(row)

        # 데이터에서 \t 문자열 제거
        # -------------------------------------------------

        characters = "\t"
        try:
            string = row[0]
        except IndexError:
            continue
        for x in range(len(characters)):
            # string = string.replace(characters[x],"")# \t 삭제한 row[0] 을 string 변수에 저장
            string = "0"
            cnt_list = cnt - 12  # temperature_realdata1[0][0]을 자동으로 카운트해주기 위해

            temperature_realdata1[cnt_list][0] = string  # temperature_realdata1[n][0]에 있는 리스트 요소를 string으로 바꾸기 위함
f2.close()
# __________________________________________________

# 필요한 타이틀만 남겨두기
# -------------------------------------------------
cnt_1 = 0

for p in temperature_realdata1:

    try:
        del temperature_realdata1[cnt_1][5]

    except IndexError:
        continue
    cnt_1 += 1

cnt_1 = 0
cnt_2 = 7

for p in temperature_realdata1:
    for x in range(2):
        try:
            del temperature_realdata1[cnt_1][cnt_2]

        except IndexError:
            continue

        cnt_2 -= 1
    cnt_2 = 7
    cnt_1 += 1

cnt_1 = 0

for p in temperature_realdata1:

    try:
        del temperature_realdata1[cnt_1][0]

    except IndexError:
        continue
    cnt_1 += 1

# __________________________________________________

# 저장된 데이터에서 비어있는 쓰레기값 제거
# -------------------------------------------------
lendata = len(temperature_realdata1) - 11

for p1 in range(11):
    try:
        del temperature_realdata1[lendata]
    except IndexError:
        continue
i = 0
k = 0
datelist = [0 for row in range(len(temperature_realdata1))]
for i in range(len(temperature_realdata1)):
    datelist[i] = temperature_realdata1[i][1]

###################################################################################################################################


procnt = 0

kindcode = ['쌀', '콩', '팥', '고구마', '감자', '시금치', '열무', '대파', ]
unitcode = ['']
colslist = [[0 for col in range(84)] for row in range(1)]

for pro in range(len(kindcode)):
    print(kindcode[pro])
    procnt += 1

    try:
        df = pd.read_excel('농산물 ' + year + kindcode[pro] + '.xlsx', header=0)
    except:
        continue
    if kindcode[pro] == '쌀':
        print(kindcode[pro], '  데이터 정제 시작')

        df_list = df.values.tolist()
        df_col = list([col for col in df])
        del df_col[0]

        pro_date = [0 for i in range(len(df_list[0]) - 1)]
        avg = [0 for i in range(len(df_list[0]) - 1)]  # 전체 평균값
        avg_fake = [0 for i in range(len(df_list[0]) - 2)]
        seoul = [0 for i in range(len(df_list[0]) - 1)]  # 서울 평균값
        busan = [0 for i in range(len(df_list[0]) - 1)]  # 부산 평균값
        daegoo = [0 for i in range(len(df_list[0]) - 1)]  # 대구 평균값
        goangjoo = [0 for i in range(len(df_list[0]) - 1)]  # 광주 평균값
        daejeon = [0 for i in range(len(df_list[0]) - 1)]  # 대전 평균값

        # --------------------------------------------------------데이터상의 날짜 구하기
        for i in range(len(df_col)):
            pp = word_tokenize(str(df_col[i]))
            z = pp[0]
            try:

                pro_date[i] = z
            except:
                continue
        i = 0

        # --------------------------------------------------------

        # ========================================================평균가격,서울,부산,대구,광주,대전 별로 리스트 분리

        for i in range(len(avg)):
            if i == 0:
                try:
                    avg = df_list[i][1:-1]
                except:
                    continue
            elif i == 1:
                continue

            elif i == 2:
                try:
                    seoul = df_list[i][1:]
                except:
                    continue
            elif i == 3:
                try:
                    busan = df_list[i][1:]
                except:
                    continue

            elif i == 4:
                try:
                    daegoo = df_list[i][1:]
                except:
                    continue
            elif i == 5:
                try:
                    goangjoo = df_list[i][1:]
                except:
                    continue
            elif i == 6:
                try:
                    daejeon = df_list[i][1:]
                except:
                    continue

        # ========================================================

        location = {'서울': 'seoul', '부산': 'busan', '대구': 'daegoo', '광주': 'goangjoo', '대전': 'daejeon'}
        pro_duct1 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct2 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct3 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct4 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct5 = [[0 for col in range(4)] for row in range(len(pro_date))]

        fakelist = [0 for row in range(len(datelist))]
        for i in range(len(pro_date)):
            fakelist[i] = pro_date[i]

        lost_date = list(set(datelist) - set(fakelist))

        sum_fake_list1 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list2 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list3 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list4 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list5 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        i = 0

        # -------------------------------------------------------------------평균가의 평균가격을 구함
        for i in range(len(avg)):
            value = avg[i].replace(',', '')
            avg[i] = int(value)

        for fake in range(len(avg_fake)):
            try:
                avg_fake[fake] = avg[fake]
            except:
                continue

        i = 0

        for i in range(len(avg_fake)):
            try:
                avg_fake[i] += avg_fake[i - 1]
                if i == (len(avg) - 1):
                    avg_fake[i] = avg_fake[i] - avg[i]
            except:
                continue

        avg_plus_avg = int((avg_fake[(len(avg_fake) - 1)]) / len(avg_fake))

        for lo in location:

            i = 0

            if lo == '서울':

                for i in range(len(seoul)):  # 서울 가격의 문자열 제거후 int로 변경
                    if seoul[i] == '-':
                        seoul[i] = seoul[i].replace('-', str(avg_plus_avg))
                    elif seoul[i] == '0':
                        seoul[i] = seoul[i].replace('0', str(avg_plus_avg))

                    value = re.sub(',', '', seoul[i])
                    seoul[i] = int(value)



            elif lo == '부산':

                for i in range(len(busan)):  # 부산 가격의 문자열 제거후 int로 변경
                    if busan[i] == '-':
                        busan[i] = busan[i].replace('-', str(avg_plus_avg))
                    elif busan[i] == '0':
                        busan[i] = busan[i].replace('0', str(avg_plus_avg))

                    value = re.sub(',', '', busan[i])
                    busan[i] = int(value)

            elif lo == '대구':

                for i in range(len(daegoo)):  # 대구 가격의 문자열 제거후 int로 변경
                    if daegoo[i] == '-':
                        daegoo[i] = daegoo[i].replace('-', str(avg_plus_avg))
                    elif daegoo[i] == '0':
                        daegoo[i] = daegoo[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', daegoo[i])
                    daegoo[i] = int(value)

            elif lo == '광주':

                for i in range(len(goangjoo)):  # 광주 가격의 문자열 제거후 int로 변경
                    if goangjoo[i] == '-':
                        goangjoo[i] = goangjoo[i].replace('-', str(avg_plus_avg))
                    elif goangjoo[i] == '0':
                        goangjoo[i] = goangjoo[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', goangjoo[i])
                    goangjoo[i] = int(value)

            elif lo == '대전':

                for i in range(len(daejeon)):  # 대전 가격의 문자열 제거후 int로 변경
                    if daejeon[i] == '-':
                        daejeon[i] = daejeon[i].replace('-', str(avg_plus_avg))
                    elif daejeon[i] == '0':
                        daejeon[i] = daejeon[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', daejeon[i])
                    daejeon[i] = int(value)

            # -------------------------------------------------------- 하나의 리스트에 농산물 정보 넣기

        i = 0
        z = 0

        for lo in location:

            if lo == '서울':
                for i in range(len(pro_duct1)):

                    for z in range(len(pro_duct1[0])):
                        if z == 0:
                            try:
                                pro_duct1[i][z] = seoul[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct1[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct1[i][z] = '서울'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct1[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list1[i][z] = seoul[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list1[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list1[i][z] = '서울'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list1[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '부산':
                for i in range(len(pro_duct2)):

                    for z in range(len(pro_duct2[0])):
                        if z == 0:
                            try:
                                pro_duct2[i][z] = busan[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct2[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct2[i][z] = '부산'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct2[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list2[i][z] = busan[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list2[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list2[i][z] = '부산'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list2[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '대구':

                for i in range(len(pro_duct3)):

                    for z in range(len(pro_duct3[0])):
                        if z == 0:
                            try:
                                pro_duct3[i][z] = daegoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct3[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct3[i][z] = '대구'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct3[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list3[i][z] = daegoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list3[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list3[i][z] = '대구'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list3[i][z] = lost_date[i]
                            except:
                                continue


            elif lo == '광주':

                for i in range(len(pro_duct4)):

                    for z in range(len(pro_duct4[0])):
                        if z == 0:
                            try:
                                pro_duct4[i][z] = goangjoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct4[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct4[i][z] = '광주'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct4[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list4[i][z] = goangjoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list4[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list4[i][z] = '광주'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list4[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '대전':

                for i in range(len(pro_duct5)):

                    for z in range(len(pro_duct5[0])):
                        if z == 0:
                            try:
                                pro_duct5[i][z] = daejeon[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct5[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct5[i][z] = '대전'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct5[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list5[i][z] = daejeon[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list5[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list5[i][z] = '대전'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list5[i][z] = lost_date[i]
                            except:
                                continue

        sum_pro_data1 = pro_duct1 + sum_fake_list1
        sum_pro_data2 = pro_duct2 + sum_fake_list2
        sum_pro_data3 = pro_duct3 + sum_fake_list3
        sum_pro_data4 = pro_duct4 + sum_fake_list4
        sum_pro_data5 = pro_duct5 + sum_fake_list5
        loca_data11 = sum_pro_data1 + sum_pro_data2 + sum_pro_data3 + sum_pro_data4 + sum_pro_data5



    elif kindcode[pro] == '콩':
        print(kindcode[pro], '  데이터 정제 시작')

        df_list = df.values.tolist()
        df_col = list([col for col in df])
        del df_col[0]

        pro_date = [0 for i in range(len(df_list[0]) - 1)]
        avg = [0 for i in range(len(df_list[0]) - 1)]  # 전체 평균값
        avg_fake = [0 for i in range(len(df_list[0]) - 2)]
        seoul = [0 for i in range(len(df_list[0]) - 1)]  # 서울 평균값
        busan = [0 for i in range(len(df_list[0]) - 1)]  # 부산 평균값
        daegoo = [0 for i in range(len(df_list[0]) - 1)]  # 대구 평균값
        goangjoo = [0 for i in range(len(df_list[0]) - 1)]  # 광주 평균값
        daejeon = [0 for i in range(len(df_list[0]) - 1)]  # 대전 평균값

        # --------------------------------------------------------데이터상의 날짜 구하기
        for i in range(len(df_col)):
            pp = word_tokenize(str(df_col[i]))
            z = pp[0]
            try:

                pro_date[i] = z
            except:
                continue
        i = 0

        # --------------------------------------------------------

        # ========================================================평균가격,서울,부산,대구,광주,대전 별로 리스트 분리

        for i in range(len(avg)):
            if i == 0:
                try:
                    avg = df_list[i][1:-1]
                except:
                    continue
            elif i == 1:
                continue

            elif i == 2:
                try:
                    seoul = df_list[i][1:]
                except:
                    continue
            elif i == 3:
                try:
                    busan = df_list[i][1:]
                except:
                    continue

            elif i == 4:
                try:
                    daegoo = df_list[i][1:]
                except:
                    continue
            elif i == 5:
                try:
                    goangjoo = df_list[i][1:]
                except:
                    continue
            elif i == 6:
                try:
                    daejeon = df_list[i][1:]
                except:
                    continue

        # ========================================================

        location = {'서울': 'seoul', '부산': 'busan', '대구': 'daegoo', '광주': 'goangjoo', '대전': 'daejeon'}
        pro_duct1 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct2 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct3 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct4 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct5 = [[0 for col in range(4)] for row in range(len(pro_date))]

        fakelist = [0 for row in range(len(datelist))]
        for i in range(len(pro_date)):
            fakelist[i] = pro_date[i]

        lost_date = list(set(datelist) - set(fakelist))

        sum_fake_list1 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list2 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list3 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list4 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list5 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        i = 0

        # -------------------------------------------------------------------평균가의 평균가격을 구함
        for i in range(len(avg)):
            value = avg[i].replace(',', '')
            avg[i] = int(value)

        for fake in range(len(avg_fake)):
            try:
                avg_fake[fake] = avg[fake]
            except:
                continue

        i = 0

        for i in range(len(avg_fake)):
            try:
                avg_fake[i] += avg_fake[i - 1]
                if i == (len(avg) - 1):
                    avg_fake[i] = avg_fake[i] - avg[i]
            except:
                continue

        avg_plus_avg = int((avg_fake[(len(avg_fake) - 1)]) / len(avg_fake))

        for lo in location:

            i = 0

            if lo == '서울':

                for i in range(len(seoul)):  # 서울 가격의 문자열 제거후 int로 변경
                    if seoul[i] == '-':
                        seoul[i] = seoul[i].replace('-', str(avg_plus_avg))
                    elif seoul[i] == '0':
                        seoul[i] = seoul[i].replace('0', str(avg_plus_avg))

                    value = re.sub(',', '', seoul[i])
                    seoul[i] = int(value)



            elif lo == '부산':

                for i in range(len(busan)):  # 부산 가격의 문자열 제거후 int로 변경
                    if busan[i] == '-':
                        busan[i] = busan[i].replace('-', str(avg_plus_avg))
                    elif busan[i] == '0':
                        busan[i] = busan[i].replace('0', str(avg_plus_avg))

                    value = re.sub(',', '', busan[i])
                    busan[i] = int(value)

            elif lo == '대구':

                for i in range(len(daegoo)):  # 대구 가격의 문자열 제거후 int로 변경
                    if daegoo[i] == '-':
                        daegoo[i] = daegoo[i].replace('-', str(avg_plus_avg))
                    elif daegoo[i] == '0':
                        daegoo[i] = daegoo[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', daegoo[i])
                    daegoo[i] = int(value)

            elif lo == '광주':

                for i in range(len(goangjoo)):  # 광주 가격의 문자열 제거후 int로 변경
                    if goangjoo[i] == '-':
                        goangjoo[i] = goangjoo[i].replace('-', str(avg_plus_avg))
                    elif goangjoo[i] == '0':
                        goangjoo[i] = goangjoo[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', goangjoo[i])
                    goangjoo[i] = int(value)

            elif lo == '대전':

                for i in range(len(daejeon)):  # 대전 가격의 문자열 제거후 int로 변경
                    if daejeon[i] == '-':
                        daejeon[i] = daejeon[i].replace('-', str(avg_plus_avg))
                    elif daejeon[i] == '0':
                        daejeon[i] = daejeon[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', daejeon[i])
                    daejeon[i] = int(value)

            # -------------------------------------------------------- 하나의 리스트에 농산물 정보 넣기

        i = 0
        z = 0

        for lo in location:

            if lo == '서울':
                for i in range(len(pro_duct1)):

                    for z in range(len(pro_duct1[0])):
                        if z == 0:
                            try:
                                pro_duct1[i][z] = seoul[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct1[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct1[i][z] = '서울'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct1[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list1[i][z] = seoul[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list1[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list1[i][z] = '서울'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list1[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '부산':
                for i in range(len(pro_duct2)):

                    for z in range(len(pro_duct2[0])):
                        if z == 0:
                            try:
                                pro_duct2[i][z] = busan[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct2[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct2[i][z] = '부산'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct2[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list2[i][z] = busan[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list2[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list2[i][z] = '부산'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list2[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '대구':

                for i in range(len(pro_duct3)):

                    for z in range(len(pro_duct3[0])):
                        if z == 0:
                            try:
                                pro_duct3[i][z] = daegoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct3[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct3[i][z] = '대구'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct3[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list3[i][z] = daegoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list3[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list3[i][z] = '대구'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list3[i][z] = lost_date[i]
                            except:
                                continue


            elif lo == '광주':

                for i in range(len(pro_duct4)):

                    for z in range(len(pro_duct4[0])):
                        if z == 0:
                            try:
                                pro_duct4[i][z] = goangjoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct4[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct4[i][z] = '광주'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct4[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list4[i][z] = goangjoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list4[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list4[i][z] = '광주'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list4[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '대전':

                for i in range(len(pro_duct5)):

                    for z in range(len(pro_duct5[0])):
                        if z == 0:
                            try:
                                pro_duct5[i][z] = daejeon[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct5[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct5[i][z] = '대전'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct5[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list5[i][z] = daejeon[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list5[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list5[i][z] = '대전'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list5[i][z] = lost_date[i]
                            except:
                                continue

        sum_pro_data1 = pro_duct1 + sum_fake_list1
        sum_pro_data2 = pro_duct2 + sum_fake_list2
        sum_pro_data3 = pro_duct3 + sum_fake_list3
        sum_pro_data4 = pro_duct4 + sum_fake_list4
        sum_pro_data5 = pro_duct5 + sum_fake_list5
        loca_data12 = sum_pro_data1 + sum_pro_data2 + sum_pro_data3 + sum_pro_data4 + sum_pro_data5


    elif kindcode[pro] == '팥':
        print(kindcode[pro], '  데이터 정제 시작')

        df_list = df.values.tolist()
        df_col = list([col for col in df])
        del df_col[0]

        pro_date = [0 for i in range(len(df_list[0]) - 1)]
        avg = [0 for i in range(len(df_list[0]) - 1)]  # 전체 평균값
        avg_fake = [0 for i in range(len(df_list[0]) - 2)]
        seoul = [0 for i in range(len(df_list[0]) - 1)]  # 서울 평균값
        busan = [0 for i in range(len(df_list[0]) - 1)]  # 부산 평균값
        daegoo = [0 for i in range(len(df_list[0]) - 1)]  # 대구 평균값
        goangjoo = [0 for i in range(len(df_list[0]) - 1)]  # 광주 평균값
        daejeon = [0 for i in range(len(df_list[0]) - 1)]  # 대전 평균값

        # --------------------------------------------------------데이터상의 날짜 구하기
        for i in range(len(df_col)):
            pp = word_tokenize(str(df_col[i]))
            z = pp[0]
            try:

                pro_date[i] = z
            except:
                continue
        i = 0

        # --------------------------------------------------------

        # ========================================================평균가격,서울,부산,대구,광주,대전 별로 리스트 분리

        for i in range(len(avg)):
            if i == 0:
                try:
                    avg = df_list[i][1:-1]
                except:
                    continue
            elif i == 1:
                continue

            elif i == 2:
                try:
                    seoul = df_list[i][1:]
                except:
                    continue
            elif i == 3:
                try:
                    busan = df_list[i][1:]
                except:
                    continue

            elif i == 4:
                try:
                    daegoo = df_list[i][1:]
                except:
                    continue
            elif i == 5:
                try:
                    goangjoo = df_list[i][1:]
                except:
                    continue
            elif i == 6:
                try:
                    daejeon = df_list[i][1:]
                except:
                    continue

        # ========================================================

        location = {'서울': 'seoul', '부산': 'busan', '대구': 'daegoo', '광주': 'goangjoo', '대전': 'daejeon'}
        pro_duct1 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct2 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct3 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct4 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct5 = [[0 for col in range(4)] for row in range(len(pro_date))]

        fakelist = [0 for row in range(len(datelist))]
        for i in range(len(pro_date)):
            fakelist[i] = pro_date[i]

        lost_date = list(set(datelist) - set(fakelist))

        sum_fake_list1 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list2 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list3 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list4 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list5 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        i = 0

        # -------------------------------------------------------------------평균가의 평균가격을 구함
        for i in range(len(avg)):
            value = avg[i].replace(',', '')
            avg[i] = int(value)

        for fake in range(len(avg_fake)):
            try:
                avg_fake[fake] = avg[fake]
            except:
                continue

        i = 0

        for i in range(len(avg_fake)):
            try:
                avg_fake[i] += avg_fake[i - 1]
                if i == (len(avg) - 1):
                    avg_fake[i] = avg_fake[i] - avg[i]
            except:
                continue

        avg_plus_avg = int((avg_fake[(len(avg_fake) - 1)]) / len(avg_fake))

        for lo in location:

            i = 0

            if lo == '서울':

                for i in range(len(seoul)):  # 서울 가격의 문자열 제거후 int로 변경
                    if seoul[i] == '-':
                        seoul[i] = seoul[i].replace('-', str(avg_plus_avg))
                    elif seoul[i] == '0':
                        seoul[i] = seoul[i].replace('0', str(avg_plus_avg))

                    value = re.sub(',', '', seoul[i])
                    seoul[i] = int(value)



            elif lo == '부산':

                for i in range(len(busan)):  # 부산 가격의 문자열 제거후 int로 변경
                    if busan[i] == '-':
                        busan[i] = busan[i].replace('-', str(avg_plus_avg))
                    elif busan[i] == '0':
                        busan[i] = busan[i].replace('0', str(avg_plus_avg))

                    value = re.sub(',', '', busan[i])
                    busan[i] = int(value)

            elif lo == '대구':

                for i in range(len(daegoo)):  # 대구 가격의 문자열 제거후 int로 변경
                    if daegoo[i] == '-':
                        daegoo[i] = daegoo[i].replace('-', str(avg_plus_avg))
                    elif daegoo[i] == '0':
                        daegoo[i] = daegoo[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', daegoo[i])
                    daegoo[i] = int(value)

            elif lo == '광주':

                for i in range(len(goangjoo)):  # 광주 가격의 문자열 제거후 int로 변경
                    if goangjoo[i] == '-':
                        goangjoo[i] = goangjoo[i].replace('-', str(avg_plus_avg))
                    elif goangjoo[i] == '0':
                        goangjoo[i] = goangjoo[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', goangjoo[i])
                    goangjoo[i] = int(value)

            elif lo == '대전':

                for i in range(len(daejeon)):  # 대전 가격의 문자열 제거후 int로 변경
                    if daejeon[i] == '-':
                        daejeon[i] = daejeon[i].replace('-', str(avg_plus_avg))
                    elif daejeon[i] == '0':
                        daejeon[i] = daejeon[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', daejeon[i])
                    daejeon[i] = int(value)

            # -------------------------------------------------------- 하나의 리스트에 농산물 정보 넣기

        i = 0
        z = 0

        for lo in location:

            if lo == '서울':
                for i in range(len(pro_duct1)):

                    for z in range(len(pro_duct1[0])):
                        if z == 0:
                            try:
                                pro_duct1[i][z] = seoul[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct1[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct1[i][z] = '서울'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct1[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list1[i][z] = seoul[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list1[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list1[i][z] = '서울'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list1[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '부산':
                for i in range(len(pro_duct2)):

                    for z in range(len(pro_duct2[0])):
                        if z == 0:
                            try:
                                pro_duct2[i][z] = busan[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct2[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct2[i][z] = '부산'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct2[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list2[i][z] = busan[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list2[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list2[i][z] = '부산'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list2[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '대구':

                for i in range(len(pro_duct3)):

                    for z in range(len(pro_duct3[0])):
                        if z == 0:
                            try:
                                pro_duct3[i][z] = daegoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct3[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct3[i][z] = '대구'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct3[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list3[i][z] = daegoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list3[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list3[i][z] = '대구'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list3[i][z] = lost_date[i]
                            except:
                                continue


            elif lo == '광주':

                for i in range(len(pro_duct4)):

                    for z in range(len(pro_duct4[0])):
                        if z == 0:
                            try:
                                pro_duct4[i][z] = goangjoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct4[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct4[i][z] = '광주'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct4[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list4[i][z] = goangjoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list4[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list4[i][z] = '광주'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list4[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '대전':

                for i in range(len(pro_duct5)):

                    for z in range(len(pro_duct5[0])):
                        if z == 0:
                            try:
                                pro_duct5[i][z] = daejeon[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct5[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct5[i][z] = '대전'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct5[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list5[i][z] = daejeon[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list5[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list5[i][z] = '대전'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list5[i][z] = lost_date[i]
                            except:
                                continue

        sum_pro_data1 = pro_duct1 + sum_fake_list1
        sum_pro_data2 = pro_duct2 + sum_fake_list2
        sum_pro_data3 = pro_duct3 + sum_fake_list3
        sum_pro_data4 = pro_duct4 + sum_fake_list4
        sum_pro_data5 = pro_duct5 + sum_fake_list5
        loca_data13 = sum_pro_data1 + sum_pro_data2 + sum_pro_data3 + sum_pro_data4 + sum_pro_data5

    elif kindcode[pro] == '고구마':
        print(kindcode[pro], '  데이터 정제 시작')

        df_list = df.values.tolist()
        df_col = list([col for col in df])
        del df_col[0]

        pro_date = [0 for i in range(len(df_list[0]) - 1)]
        avg = [0 for i in range(len(df_list[0]) - 1)]  # 전체 평균값
        avg_fake = [0 for i in range(len(df_list[0]) - 2)]
        seoul = [0 for i in range(len(df_list[0]) - 1)]  # 서울 평균값
        busan = [0 for i in range(len(df_list[0]) - 1)]  # 부산 평균값
        daegoo = [0 for i in range(len(df_list[0]) - 1)]  # 대구 평균값
        goangjoo = [0 for i in range(len(df_list[0]) - 1)]  # 광주 평균값
        daejeon = [0 for i in range(len(df_list[0]) - 1)]  # 대전 평균값

        # --------------------------------------------------------데이터상의 날짜 구하기
        for i in range(len(df_col)):
            pp = word_tokenize(str(df_col[i]))
            z = pp[0]
            try:

                pro_date[i] = z
            except:
                continue
        i = 0

        # --------------------------------------------------------

        # ========================================================평균가격,서울,부산,대구,광주,대전 별로 리스트 분리

        for i in range(len(avg)):
            if i == 0:
                try:
                    avg = df_list[i][1:-1]
                except:
                    continue
            elif i == 1:
                continue

            elif i == 2:
                try:
                    seoul = df_list[i][1:]
                except:
                    continue
            elif i == 3:
                try:
                    busan = df_list[i][1:]
                except:
                    continue

            elif i == 4:
                try:
                    daegoo = df_list[i][1:]
                except:
                    continue
            elif i == 5:
                try:
                    goangjoo = df_list[i][1:]
                except:
                    continue
            elif i == 6:
                try:
                    daejeon = df_list[i][1:]
                except:
                    continue

        # ========================================================

        location = {'서울': 'seoul', '부산': 'busan', '대구': 'daegoo', '광주': 'goangjoo', '대전': 'daejeon'}
        pro_duct1 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct2 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct3 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct4 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct5 = [[0 for col in range(4)] for row in range(len(pro_date))]

        fakelist = [0 for row in range(len(datelist))]
        for i in range(len(pro_date)):
            fakelist[i] = pro_date[i]

        lost_date = list(set(datelist) - set(fakelist))

        sum_fake_list1 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list2 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list3 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list4 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list5 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        i = 0

        # -------------------------------------------------------------------평균가의 평균가격을 구함
        for i in range(len(avg)):
            value = avg[i].replace(',', '')

            avg[i] = int(value)

        for fake in range(len(avg_fake)):
            try:
                avg_fake[fake] = avg[fake]
            except:
                continue

        i = 0

        for i in range(len(avg_fake)):
            try:
                avg_fake[i] += avg_fake[i - 1]
                if i == (len(avg) - 1):
                    avg_fake[i] = avg_fake[i] - avg[i]
            except:
                continue

        avg_plus_avg = int((avg_fake[(len(avg_fake) - 1)]) / len(avg_fake))

        for lo in location:

            i = 0

            if lo == '서울':

                for i in range(len(seoul)):  # 서울 가격의 문자열 제거후 int로 변경
                    if seoul[i] == '-':
                        seoul[i] = seoul[i].replace('-', str(avg_plus_avg))
                    elif seoul[i] == '0':
                        seoul[i] = seoul[i].replace('0', str(avg_plus_avg))

                    value = re.sub(',', '', seoul[i])
                    seoul[i] = int(value)



            elif lo == '부산':

                for i in range(len(busan)):  # 부산 가격의 문자열 제거후 int로 변경
                    if busan[i] == '-':
                        busan[i] = busan[i].replace('-', str(avg_plus_avg))
                    elif busan[i] == '0':
                        busan[i] = busan[i].replace('0', str(avg_plus_avg))

                    value = re.sub(',', '', busan[i])
                    busan[i] = int(value)

            elif lo == '대구':

                for i in range(len(daegoo)):  # 대구 가격의 문자열 제거후 int로 변경
                    if daegoo[i] == '-':
                        daegoo[i] = daegoo[i].replace('-', str(avg_plus_avg))
                    elif daegoo[i] == '0':
                        daegoo[i] = daegoo[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', daegoo[i])
                    daegoo[i] = int(value)

            elif lo == '광주':

                for i in range(len(goangjoo)):  # 광주 가격의 문자열 제거후 int로 변경
                    if goangjoo[i] == '-':
                        goangjoo[i] = goangjoo[i].replace('-', str(avg_plus_avg))
                    elif goangjoo[i] == '0':
                        goangjoo[i] = goangjoo[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', goangjoo[i])
                    goangjoo[i] = int(value)

            elif lo == '대전':

                for i in range(len(daejeon)):  # 대전 가격의 문자열 제거후 int로 변경
                    if daejeon[i] == '-':
                        daejeon[i] = daejeon[i].replace('-', str(avg_plus_avg))
                    elif daejeon[i] == '0':
                        daejeon[i] = daejeon[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', daejeon[i])
                    daejeon[i] = int(value)

            # -------------------------------------------------------- 하나의 리스트에 농산물 정보 넣기

        i = 0
        z = 0

        for lo in location:

            if lo == '서울':
                for i in range(len(pro_duct1)):

                    for z in range(len(pro_duct1[0])):
                        if z == 0:
                            try:
                                pro_duct1[i][z] = seoul[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct1[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct1[i][z] = '서울'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct1[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list1[i][z] = seoul[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list1[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list1[i][z] = '서울'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list1[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '부산':
                for i in range(len(pro_duct2)):

                    for z in range(len(pro_duct2[0])):
                        if z == 0:
                            try:
                                pro_duct2[i][z] = busan[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct2[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct2[i][z] = '부산'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct2[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list2[i][z] = busan[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list2[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list2[i][z] = '부산'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list2[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '대구':

                for i in range(len(pro_duct3)):

                    for z in range(len(pro_duct3[0])):
                        if z == 0:
                            try:
                                pro_duct3[i][z] = daegoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct3[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct3[i][z] = '대구'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct3[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list3[i][z] = daegoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list3[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list3[i][z] = '대구'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list3[i][z] = lost_date[i]
                            except:
                                continue


            elif lo == '광주':

                for i in range(len(pro_duct4)):

                    for z in range(len(pro_duct4[0])):
                        if z == 0:
                            try:
                                pro_duct4[i][z] = goangjoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct4[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct4[i][z] = '광주'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct4[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list4[i][z] = goangjoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list4[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list4[i][z] = '광주'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list4[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '대전':

                for i in range(len(pro_duct5)):

                    for z in range(len(pro_duct5[0])):
                        if z == 0:
                            try:
                                pro_duct5[i][z] = daejeon[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct5[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct5[i][z] = '대전'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct5[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list5[i][z] = daejeon[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list5[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list5[i][z] = '대전'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list5[i][z] = lost_date[i]
                            except:
                                continue

        sum_pro_data1 = pro_duct1 + sum_fake_list1
        sum_pro_data2 = pro_duct2 + sum_fake_list2
        sum_pro_data3 = pro_duct3 + sum_fake_list3
        sum_pro_data4 = pro_duct4 + sum_fake_list4
        sum_pro_data5 = pro_duct5 + sum_fake_list5
        loca_data14 = sum_pro_data1 + sum_pro_data2 + sum_pro_data3 + sum_pro_data4 + sum_pro_data5



    elif kindcode[pro] == '감자':
        print(kindcode[pro], '  데이터 정제 시작')

        df_list = df.values.tolist()
        df_col = list([col for col in df])
        del df_col[0]

        pro_date = [0 for i in range(len(df_list[0]) - 1)]
        avg = [0 for i in range(len(df_list[0]) - 1)]  # 전체 평균값
        avg_fake = [0 for i in range(len(df_list[0]) - 2)]
        seoul = [0 for i in range(len(df_list[0]) - 1)]  # 서울 평균값
        busan = [0 for i in range(len(df_list[0]) - 1)]  # 부산 평균값
        daegoo = [0 for i in range(len(df_list[0]) - 1)]  # 대구 평균값
        goangjoo = [0 for i in range(len(df_list[0]) - 1)]  # 광주 평균값
        daejeon = [0 for i in range(len(df_list[0]) - 1)]  # 대전 평균값

        # --------------------------------------------------------데이터상의 날짜 구하기
        for i in range(len(df_col)):
            pp = word_tokenize(str(df_col[i]))
            z = pp[0]
            try:

                pro_date[i] = z
            except:
                continue
        i = 0

        # --------------------------------------------------------

        # ========================================================평균가격,서울,부산,대구,광주,대전 별로 리스트 분리

        for i in range(len(avg)):
            if i == 0:
                try:
                    avg = df_list[i][1:-1]
                except:
                    continue
            elif i == 1:
                continue

            elif i == 2:
                try:
                    seoul = df_list[i][1:]
                except:
                    continue
            elif i == 3:
                try:
                    busan = df_list[i][1:]
                except:
                    continue

            elif i == 4:
                try:
                    daegoo = df_list[i][1:]
                except:
                    continue
            elif i == 5:
                try:
                    goangjoo = df_list[i][1:]
                except:
                    continue
            elif i == 6:
                try:
                    daejeon = df_list[i][1:]
                except:
                    continue

        # ========================================================

        location = {'서울': 'seoul', '부산': 'busan', '대구': 'daegoo', '광주': 'goangjoo', '대전': 'daejeon'}
        pro_duct1 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct2 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct3 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct4 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct5 = [[0 for col in range(4)] for row in range(len(pro_date))]

        fakelist = [0 for row in range(len(datelist))]
        for i in range(len(pro_date)):
            fakelist[i] = pro_date[i]

        lost_date = list(set(datelist) - set(fakelist))

        sum_fake_list1 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list2 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list3 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list4 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list5 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        i = 0

        # -------------------------------------------------------------------평균가의 평균가격을 구함
        for i in range(len(avg)):
            value = avg[i].replace(',', '')
            avg[i] = int(value)

        for fake in range(len(avg_fake)):
            try:
                avg_fake[fake] = avg[fake]
            except:
                continue

        i = 0

        for i in range(len(avg_fake)):
            try:
                avg_fake[i] += avg_fake[i - 1]
                if i == (len(avg) - 1):
                    avg_fake[i] = avg_fake[i] - avg[i]
            except:
                continue

        avg_plus_avg = int((avg_fake[(len(avg_fake) - 1)]) / len(avg_fake))

        for lo in location:

            i = 0

            if lo == '서울':

                for i in range(len(seoul)):  # 서울 가격의 문자열 제거후 int로 변경
                    if seoul[i] == '-':
                        seoul[i] = seoul[i].replace('-', str(avg_plus_avg))
                    elif seoul[i] == '0':
                        seoul[i] = seoul[i].replace('0', str(avg_plus_avg))

                    value = re.sub(',', '', seoul[i])
                    seoul[i] = int(value)



            elif lo == '부산':

                for i in range(len(busan)):  # 부산 가격의 문자열 제거후 int로 변경
                    if busan[i] == '-':
                        busan[i] = busan[i].replace('-', str(avg_plus_avg))
                    elif busan[i] == '0':
                        busan[i] = busan[i].replace('0', str(avg_plus_avg))

                    value = re.sub(',', '', busan[i])
                    busan[i] = int(value)

            elif lo == '대구':

                for i in range(len(daegoo)):  # 대구 가격의 문자열 제거후 int로 변경
                    if daegoo[i] == '-':
                        daegoo[i] = daegoo[i].replace('-', str(avg_plus_avg))
                    elif daegoo[i] == '0':
                        daegoo[i] = daegoo[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', daegoo[i])
                    daegoo[i] = int(value)

            elif lo == '광주':

                for i in range(len(goangjoo)):  # 광주 가격의 문자열 제거후 int로 변경
                    if goangjoo[i] == '-':
                        goangjoo[i] = goangjoo[i].replace('-', str(avg_plus_avg))
                    elif goangjoo[i] == '0':
                        goangjoo[i] = goangjoo[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', goangjoo[i])
                    goangjoo[i] = int(value)

            elif lo == '대전':

                for i in range(len(daejeon)):  # 대전 가격의 문자열 제거후 int로 변경
                    if daejeon[i] == '-':
                        daejeon[i] = daejeon[i].replace('-', str(avg_plus_avg))
                    elif daejeon[i] == '0':
                        daejeon[i] = daejeon[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', daejeon[i])
                    daejeon[i] = int(value)

            # -------------------------------------------------------- 하나의 리스트에 농산물 정보 넣기

        i = 0
        z = 0

        for lo in location:

            if lo == '서울':
                for i in range(len(pro_duct1)):

                    for z in range(len(pro_duct1[0])):
                        if z == 0:
                            try:
                                pro_duct1[i][z] = seoul[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct1[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct1[i][z] = '서울'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct1[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list1[i][z] = seoul[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list1[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list1[i][z] = '서울'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list1[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '부산':
                for i in range(len(pro_duct2)):

                    for z in range(len(pro_duct2[0])):
                        if z == 0:
                            try:
                                pro_duct2[i][z] = busan[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct2[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct2[i][z] = '부산'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct2[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list2[i][z] = busan[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list2[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list2[i][z] = '부산'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list2[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '대구':

                for i in range(len(pro_duct3)):

                    for z in range(len(pro_duct3[0])):
                        if z == 0:
                            try:
                                pro_duct3[i][z] = daegoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct3[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct3[i][z] = '대구'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct3[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list3[i][z] = daegoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list3[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list3[i][z] = '대구'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list3[i][z] = lost_date[i]
                            except:
                                continue


            elif lo == '광주':

                for i in range(len(pro_duct4)):

                    for z in range(len(pro_duct4[0])):
                        if z == 0:
                            try:
                                pro_duct4[i][z] = goangjoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct4[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct4[i][z] = '광주'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct4[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list4[i][z] = goangjoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list4[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list4[i][z] = '광주'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list4[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '대전':

                for i in range(len(pro_duct5)):

                    for z in range(len(pro_duct5[0])):
                        if z == 0:
                            try:
                                pro_duct5[i][z] = daejeon[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct5[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct5[i][z] = '대전'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct5[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list5[i][z] = daejeon[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list5[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list5[i][z] = '대전'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list5[i][z] = lost_date[i]
                            except:
                                continue

        sum_pro_data1 = pro_duct1 + sum_fake_list1
        sum_pro_data2 = pro_duct2 + sum_fake_list2
        sum_pro_data3 = pro_duct3 + sum_fake_list3
        sum_pro_data4 = pro_duct4 + sum_fake_list4
        sum_pro_data5 = pro_duct5 + sum_fake_list5
        loca_data15 = sum_pro_data1 + sum_pro_data2 + sum_pro_data3 + sum_pro_data4 + sum_pro_data5


    elif kindcode[pro] == '시금치':
        print(kindcode[pro], '  데이터 정제 시작')

        df_list = df.values.tolist()
        df_col = list([col for col in df])
        del df_col[0]

        pro_date = [0 for i in range(len(df_list[0]) - 1)]
        avg = [0 for i in range(len(df_list[0]) - 1)]  # 전체 평균값
        avg_fake = [0 for i in range(len(df_list[0]) - 2)]
        seoul = [0 for i in range(len(df_list[0]) - 1)]  # 서울 평균값
        busan = [0 for i in range(len(df_list[0]) - 1)]  # 부산 평균값
        daegoo = [0 for i in range(len(df_list[0]) - 1)]  # 대구 평균값
        goangjoo = [0 for i in range(len(df_list[0]) - 1)]  # 광주 평균값
        daejeon = [0 for i in range(len(df_list[0]) - 1)]  # 대전 평균값

        # --------------------------------------------------------데이터상의 날짜 구하기
        for i in range(len(df_col)):
            pp = word_tokenize(str(df_col[i]))
            z = pp[0]
            try:

                pro_date[i] = z
            except:
                continue
        i = 0

        # --------------------------------------------------------

        # ========================================================평균가격,서울,부산,대구,광주,대전 별로 리스트 분리

        for i in range(len(avg)):
            if i == 0:
                try:
                    avg = df_list[i][1:-1]
                except:
                    continue
            elif i == 1:
                continue

            elif i == 2:
                try:
                    seoul = df_list[i][1:]
                except:
                    continue
            elif i == 3:
                try:
                    busan = df_list[i][1:]
                except:
                    continue

            elif i == 4:
                try:
                    daegoo = df_list[i][1:]
                except:
                    continue
            elif i == 5:
                try:
                    goangjoo = df_list[i][1:]
                except:
                    continue
            elif i == 6:
                try:
                    daejeon = df_list[i][1:]
                except:
                    continue

        # ========================================================

        location = {'서울': 'seoul', '부산': 'busan', '대구': 'daegoo', '광주': 'goangjoo', '대전': 'daejeon'}
        pro_duct1 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct2 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct3 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct4 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct5 = [[0 for col in range(4)] for row in range(len(pro_date))]

        fakelist = [0 for row in range(len(datelist))]
        for i in range(len(pro_date)):
            fakelist[i] = pro_date[i]

        lost_date = list(set(datelist) - set(fakelist))

        sum_fake_list1 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list2 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list3 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list4 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list5 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        i = 0

        # -------------------------------------------------------------------평균가의 평균가격을 구함
        for i in range(len(avg)):
            value = avg[i].replace(',', '')
            avg[i] = int(value)

        for fake in range(len(avg_fake)):
            try:
                avg_fake[fake] = avg[fake]
            except:
                continue

        i = 0

        for i in range(len(avg_fake)):
            try:
                avg_fake[i] += avg_fake[i - 1]
                if i == (len(avg) - 1):
                    avg_fake[i] = avg_fake[i] - avg[i]
            except:
                continue

        avg_plus_avg = int((avg_fake[(len(avg_fake) - 1)]) / len(avg_fake))

        for lo in location:

            i = 0

            if lo == '서울':

                for i in range(len(seoul)):  # 서울 가격의 문자열 제거후 int로 변경
                    if seoul[i] == '-':
                        seoul[i] = seoul[i].replace('-', str(avg_plus_avg))
                    elif seoul[i] == '0':
                        seoul[i] = seoul[i].replace('0', str(avg_plus_avg))

                    value = re.sub(',', '', seoul[i])
                    seoul[i] = int(value)



            elif lo == '부산':

                for i in range(len(busan)):  # 부산 가격의 문자열 제거후 int로 변경
                    if busan[i] == '-':
                        busan[i] = busan[i].replace('-', str(avg_plus_avg))
                    elif busan[i] == '0':
                        busan[i] = busan[i].replace('0', str(avg_plus_avg))

                    value = re.sub(',', '', busan[i])
                    busan[i] = int(value)

            elif lo == '대구':

                for i in range(len(daegoo)):  # 대구 가격의 문자열 제거후 int로 변경
                    if daegoo[i] == '-':
                        daegoo[i] = daegoo[i].replace('-', str(avg_plus_avg))
                    elif daegoo[i] == '0':
                        daegoo[i] = daegoo[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', daegoo[i])
                    daegoo[i] = int(value)

            elif lo == '광주':

                for i in range(len(goangjoo)):  # 광주 가격의 문자열 제거후 int로 변경
                    if goangjoo[i] == '-':
                        goangjoo[i] = goangjoo[i].replace('-', str(avg_plus_avg))
                    elif goangjoo[i] == '0':
                        goangjoo[i] = goangjoo[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', goangjoo[i])
                    goangjoo[i] = int(value)

            elif lo == '대전':

                for i in range(len(daejeon)):  # 대전 가격의 문자열 제거후 int로 변경
                    if daejeon[i] == '-':
                        daejeon[i] = daejeon[i].replace('-', str(avg_plus_avg))
                    elif daejeon[i] == '0':
                        daejeon[i] = daejeon[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', daejeon[i])
                    daejeon[i] = int(value)

            # -------------------------------------------------------- 하나의 리스트에 농산물 정보 넣기

        i = 0
        z = 0

        for lo in location:

            if lo == '서울':
                for i in range(len(pro_duct1)):

                    for z in range(len(pro_duct1[0])):
                        if z == 0:
                            try:
                                pro_duct1[i][z] = seoul[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct1[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct1[i][z] = '서울'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct1[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list1[i][z] = seoul[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list1[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list1[i][z] = '서울'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list1[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '부산':
                for i in range(len(pro_duct2)):

                    for z in range(len(pro_duct2[0])):
                        if z == 0:
                            try:
                                pro_duct2[i][z] = busan[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct2[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct2[i][z] = '부산'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct2[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list2[i][z] = busan[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list2[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list2[i][z] = '부산'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list2[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '대구':

                for i in range(len(pro_duct3)):

                    for z in range(len(pro_duct3[0])):
                        if z == 0:
                            try:
                                pro_duct3[i][z] = daegoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct3[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct3[i][z] = '대구'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct3[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list3[i][z] = daegoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list3[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list3[i][z] = '대구'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list3[i][z] = lost_date[i]
                            except:
                                continue


            elif lo == '광주':

                for i in range(len(pro_duct4)):

                    for z in range(len(pro_duct4[0])):
                        if z == 0:
                            try:
                                pro_duct4[i][z] = goangjoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct4[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct4[i][z] = '광주'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct4[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list4[i][z] = goangjoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list4[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list4[i][z] = '광주'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list4[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '대전':

                for i in range(len(pro_duct5)):

                    for z in range(len(pro_duct5[0])):
                        if z == 0:
                            try:
                                pro_duct5[i][z] = daejeon[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct5[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct5[i][z] = '대전'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct5[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list5[i][z] = daejeon[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list5[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list5[i][z] = '대전'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list5[i][z] = lost_date[i]
                            except:
                                continue

        sum_pro_data1 = pro_duct1 + sum_fake_list1
        sum_pro_data2 = pro_duct2 + sum_fake_list2
        sum_pro_data3 = pro_duct3 + sum_fake_list3
        sum_pro_data4 = pro_duct4 + sum_fake_list4
        sum_pro_data5 = pro_duct5 + sum_fake_list5
        loca_data16 = sum_pro_data1 + sum_pro_data2 + sum_pro_data3 + sum_pro_data4 + sum_pro_data5


    elif kindcode[pro] == '열무':
        print(kindcode[pro], '  데이터 정제 시작')

        df_list = df.values.tolist()
        df_col = list([col for col in df])
        del df_col[0]

        pro_date = [0 for i in range(len(df_list[0]) - 1)]
        avg = [0 for i in range(len(df_list[0]) - 1)]  # 전체 평균값
        avg_fake = [0 for i in range(len(df_list[0]) - 2)]
        seoul = [0 for i in range(len(df_list[0]) - 1)]  # 서울 평균값
        busan = [0 for i in range(len(df_list[0]) - 1)]  # 부산 평균값
        daegoo = [0 for i in range(len(df_list[0]) - 1)]  # 대구 평균값
        goangjoo = [0 for i in range(len(df_list[0]) - 1)]  # 광주 평균값
        daejeon = [0 for i in range(len(df_list[0]) - 1)]  # 대전 평균값

        # --------------------------------------------------------데이터상의 날짜 구하기
        for i in range(len(df_col)):
            pp = word_tokenize(str(df_col[i]))
            z = pp[0]
            try:

                pro_date[i] = z
            except:
                continue
        i = 0

        # --------------------------------------------------------

        # ========================================================평균가격,서울,부산,대구,광주,대전 별로 리스트 분리

        for i in range(len(avg)):
            if i == 0:
                try:
                    avg = df_list[i][1:-1]
                except:
                    continue
            elif i == 1:
                continue

            elif i == 2:
                try:
                    seoul = df_list[i][1:]
                except:
                    continue
            elif i == 3:
                try:
                    busan = df_list[i][1:]
                except:
                    continue

            elif i == 4:
                try:
                    daegoo = df_list[i][1:]
                except:
                    continue
            elif i == 5:
                try:
                    goangjoo = df_list[i][1:]
                except:
                    continue
            elif i == 6:
                try:
                    daejeon = df_list[i][1:]
                except:
                    continue

        # ========================================================

        location = {'서울': 'seoul', '부산': 'busan', '대구': 'daegoo', '광주': 'goangjoo', '대전': 'daejeon'}
        pro_duct1 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct2 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct3 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct4 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct5 = [[0 for col in range(4)] for row in range(len(pro_date))]

        fakelist = [0 for row in range(len(datelist))]
        for i in range(len(pro_date)):
            fakelist[i] = pro_date[i]

        lost_date = list(set(datelist) - set(fakelist))

        sum_fake_list1 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list2 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list3 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list4 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list5 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        i = 0

        # -------------------------------------------------------------------평균가의 평균가격을 구함
        for i in range(len(avg)):
            value = avg[i].replace(',', '')
            avg[i] = int(value)

        for fake in range(len(avg_fake)):
            try:
                avg_fake[fake] = avg[fake]
            except:
                continue

        i = 0

        for i in range(len(avg_fake)):
            try:
                avg_fake[i] += avg_fake[i - 1]
                if i == (len(avg) - 1):
                    avg_fake[i] = avg_fake[i] - avg[i]
            except:
                continue

        avg_plus_avg = int((avg_fake[(len(avg_fake) - 1)]) / len(avg_fake))

        for lo in location:

            i = 0

            if lo == '서울':

                for i in range(len(seoul)):  # 서울 가격의 문자열 제거후 int로 변경
                    if seoul[i] == '-':
                        seoul[i] = seoul[i].replace('-', str(avg_plus_avg))
                    elif seoul[i] == '0':
                        seoul[i] = seoul[i].replace('0', str(avg_plus_avg))

                    value = re.sub(',', '', seoul[i])
                    seoul[i] = int(value)



            elif lo == '부산':

                for i in range(len(busan)):  # 부산 가격의 문자열 제거후 int로 변경
                    if busan[i] == '-':
                        busan[i] = busan[i].replace('-', str(avg_plus_avg))
                    elif busan[i] == '0':
                        busan[i] = busan[i].replace('0', str(avg_plus_avg))

                    value = re.sub(',', '', busan[i])
                    busan[i] = int(value)

            elif lo == '대구':

                for i in range(len(daegoo)):  # 대구 가격의 문자열 제거후 int로 변경
                    if daegoo[i] == '-':
                        daegoo[i] = daegoo[i].replace('-', str(avg_plus_avg))
                    elif daegoo[i] == '0':
                        daegoo[i] = daegoo[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', daegoo[i])
                    daegoo[i] = int(value)

            elif lo == '광주':

                for i in range(len(goangjoo)):  # 광주 가격의 문자열 제거후 int로 변경
                    if goangjoo[i] == '-':
                        goangjoo[i] = goangjoo[i].replace('-', str(avg_plus_avg))
                    elif goangjoo[i] == '0':
                        goangjoo[i] = goangjoo[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', goangjoo[i])
                    goangjoo[i] = int(value)

            elif lo == '대전':

                for i in range(len(daejeon)):  # 대전 가격의 문자열 제거후 int로 변경
                    if daejeon[i] == '-':
                        daejeon[i] = daejeon[i].replace('-', str(avg_plus_avg))
                    elif daejeon[i] == '0':
                        daejeon[i] = daejeon[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', daejeon[i])
                    daejeon[i] = int(value)

            # -------------------------------------------------------- 하나의 리스트에 농산물 정보 넣기

        i = 0
        z = 0

        for lo in location:

            if lo == '서울':
                for i in range(len(pro_duct1)):

                    for z in range(len(pro_duct1[0])):
                        if z == 0:
                            try:
                                pro_duct1[i][z] = seoul[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct1[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct1[i][z] = '서울'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct1[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list1[i][z] = seoul[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list1[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list1[i][z] = '서울'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list1[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '부산':
                for i in range(len(pro_duct2)):

                    for z in range(len(pro_duct2[0])):
                        if z == 0:
                            try:
                                pro_duct2[i][z] = busan[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct2[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct2[i][z] = '부산'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct2[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list2[i][z] = busan[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list2[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list2[i][z] = '부산'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list2[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '대구':

                for i in range(len(pro_duct3)):

                    for z in range(len(pro_duct3[0])):
                        if z == 0:
                            try:
                                pro_duct3[i][z] = daegoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct3[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct3[i][z] = '대구'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct3[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list3[i][z] = daegoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list3[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list3[i][z] = '대구'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list3[i][z] = lost_date[i]
                            except:
                                continue


            elif lo == '광주':

                for i in range(len(pro_duct4)):

                    for z in range(len(pro_duct4[0])):
                        if z == 0:
                            try:
                                pro_duct4[i][z] = goangjoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct4[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct4[i][z] = '광주'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct4[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list4[i][z] = goangjoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list4[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list4[i][z] = '광주'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list4[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '대전':

                for i in range(len(pro_duct5)):

                    for z in range(len(pro_duct5[0])):
                        if z == 0:
                            try:
                                pro_duct5[i][z] = daejeon[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct5[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct5[i][z] = '대전'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct5[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list5[i][z] = daejeon[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list5[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list5[i][z] = '대전'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list5[i][z] = lost_date[i]
                            except:
                                continue

        sum_pro_data1 = pro_duct1 + sum_fake_list1
        sum_pro_data2 = pro_duct2 + sum_fake_list2
        sum_pro_data3 = pro_duct3 + sum_fake_list3
        sum_pro_data4 = pro_duct4 + sum_fake_list4
        sum_pro_data5 = pro_duct5 + sum_fake_list5
        loca_data17 = sum_pro_data1 + sum_pro_data2 + sum_pro_data3 + sum_pro_data4 + sum_pro_data5





    elif kindcode[pro] == '대파':
        print(kindcode[pro], '  데이터 정제 시작')

        df_list = df.values.tolist()
        df_col = list([col for col in df])
        del df_col[0]

        pro_date = [0 for i in range(len(df_list[0]) - 1)]
        avg = [0 for i in range(len(df_list[0]) - 1)]  # 전체 평균값
        avg_fake = [0 for i in range(len(df_list[0]) - 2)]
        seoul = [0 for i in range(len(df_list[0]) - 1)]  # 서울 평균값
        busan = [0 for i in range(len(df_list[0]) - 1)]  # 부산 평균값
        daegoo = [0 for i in range(len(df_list[0]) - 1)]  # 대구 평균값
        goangjoo = [0 for i in range(len(df_list[0]) - 1)]  # 광주 평균값
        daejeon = [0 for i in range(len(df_list[0]) - 1)]  # 대전 평균값

        # --------------------------------------------------------데이터상의 날짜 구하기
        for i in range(len(df_col)):
            pp = word_tokenize(str(df_col[i]))
            z = pp[0]
            try:

                pro_date[i] = z
            except:
                continue
        i = 0

        # --------------------------------------------------------

        # ========================================================평균가격,서울,부산,대구,광주,대전 별로 리스트 분리

        for i in range(len(avg)):
            if i == 0:
                try:
                    avg = df_list[i][1:-1]
                except:
                    continue
            elif i == 1:
                continue

            elif i == 2:
                try:
                    seoul = df_list[i][1:]
                except:
                    continue
            elif i == 3:
                try:
                    busan = df_list[i][1:]
                except:
                    continue

            elif i == 4:
                try:
                    daegoo = df_list[i][1:]
                except:
                    continue
            elif i == 5:
                try:
                    goangjoo = df_list[i][1:]
                except:
                    continue
            elif i == 6:
                try:
                    daejeon = df_list[i][1:]
                except:
                    continue

        # ========================================================

        location = {'서울': 'seoul', '부산': 'busan', '대구': 'daegoo', '광주': 'goangjoo', '대전': 'daejeon'}
        pro_duct1 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct2 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct3 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct4 = [[0 for col in range(4)] for row in range(len(pro_date))]
        pro_duct5 = [[0 for col in range(4)] for row in range(len(pro_date))]

        fakelist = [0 for row in range(len(datelist))]
        for i in range(len(pro_date)):
            fakelist[i] = pro_date[i]

        lost_date = list(set(datelist) - set(fakelist))

        sum_fake_list1 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list2 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list3 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list4 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        sum_fake_list5 = [[0 for col in range(4)] for row in range(len(lost_date))]  # 잃어버린 날짜에 농산물 데이터를 넣어줄 이중리스트
        i = 0

        # -------------------------------------------------------------------평균가의 평균가격을 구함
        for i in range(len(avg)):
            value = avg[i].replace(',', '')
            avg[i] = int(value)

        for fake in range(len(avg_fake)):
            try:
                avg_fake[fake] = avg[fake]
            except:
                continue

        i = 0

        for i in range(len(avg_fake)):
            try:
                avg_fake[i] += avg_fake[i - 1]
                if i == (len(avg) - 1):
                    avg_fake[i] = avg_fake[i] - avg[i]
            except:
                continue

        avg_plus_avg = int((avg_fake[(len(avg_fake) - 1)]) / len(avg_fake))

        for lo in location:

            i = 0

            if lo == '서울':

                for i in range(len(seoul)):  # 서울 가격의 문자열 제거후 int로 변경
                    if seoul[i] == '-':
                        seoul[i] = seoul[i].replace('-', str(avg_plus_avg))
                    elif seoul[i] == '0':
                        seoul[i] = seoul[i].replace('0', str(avg_plus_avg))

                    value = re.sub(',', '', seoul[i])
                    seoul[i] = int(value)



            elif lo == '부산':

                for i in range(len(busan)):  # 부산 가격의 문자열 제거후 int로 변경
                    if busan[i] == '-':
                        busan[i] = busan[i].replace('-', str(avg_plus_avg))
                    elif busan[i] == '0':
                        busan[i] = busan[i].replace('0', str(avg_plus_avg))

                    value = re.sub(',', '', busan[i])
                    busan[i] = int(value)

            elif lo == '대구':

                for i in range(len(daegoo)):  # 대구 가격의 문자열 제거후 int로 변경
                    if daegoo[i] == '-':
                        daegoo[i] = daegoo[i].replace('-', str(avg_plus_avg))
                    elif daegoo[i] == '0':
                        daegoo[i] = daegoo[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', daegoo[i])
                    daegoo[i] = int(value)

            elif lo == '광주':

                for i in range(len(goangjoo)):  # 광주 가격의 문자열 제거후 int로 변경
                    if goangjoo[i] == '-':
                        goangjoo[i] = goangjoo[i].replace('-', str(avg_plus_avg))
                    elif goangjoo[i] == '0':
                        goangjoo[i] = goangjoo[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', goangjoo[i])
                    goangjoo[i] = int(value)

            elif lo == '대전':

                for i in range(len(daejeon)):  # 대전 가격의 문자열 제거후 int로 변경
                    if daejeon[i] == '-':
                        daejeon[i] = daejeon[i].replace('-', str(avg_plus_avg))
                    elif daejeon[i] == '0':
                        daejeon[i] = daejeon[i].replace('0', str(avg_plus_avg))
                    value = re.sub(',', '', daejeon[i])
                    daejeon[i] = int(value)

            # -------------------------------------------------------- 하나의 리스트에 농산물 정보 넣기

        i = 0
        z = 0

        for lo in location:

            if lo == '서울':
                for i in range(len(pro_duct1)):

                    for z in range(len(pro_duct1[0])):
                        if z == 0:
                            try:
                                pro_duct1[i][z] = seoul[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct1[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct1[i][z] = '서울'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct1[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list1[i][z] = seoul[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list1[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list1[i][z] = '서울'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list1[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '부산':
                for i in range(len(pro_duct2)):

                    for z in range(len(pro_duct2[0])):
                        if z == 0:
                            try:
                                pro_duct2[i][z] = busan[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct2[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct2[i][z] = '부산'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct2[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list2[i][z] = busan[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list2[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list2[i][z] = '부산'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list2[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '대구':

                for i in range(len(pro_duct3)):

                    for z in range(len(pro_duct3[0])):
                        if z == 0:
                            try:
                                pro_duct3[i][z] = daegoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct3[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct3[i][z] = '대구'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct3[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list3[i][z] = daegoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list3[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list3[i][z] = '대구'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list3[i][z] = lost_date[i]
                            except:
                                continue


            elif lo == '광주':

                for i in range(len(pro_duct4)):

                    for z in range(len(pro_duct4[0])):
                        if z == 0:
                            try:
                                pro_duct4[i][z] = goangjoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct4[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct4[i][z] = '광주'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct4[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list4[i][z] = goangjoo[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list4[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list4[i][z] = '광주'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list4[i][z] = lost_date[i]
                            except:
                                continue

            elif lo == '대전':

                for i in range(len(pro_duct5)):

                    for z in range(len(pro_duct5[0])):
                        if z == 0:
                            try:
                                pro_duct5[i][z] = daejeon[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                pro_duct5[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:

                            try:
                                pro_duct5[i][z] = '대전'
                            except:
                                continue


                        elif z == 3:
                            try:
                                pro_duct5[i][z] = pro_date[i]
                            except:
                                continue

                for i in range(len(lost_date)):  # 잃어버린 날짜에 값을 넣어주기 위해 잃어버린 날짜만큼 반복문

                    for z in range(len(lost_date[0])):  # 날짜와 작물을 합치기 위한 [4][잃어버린 날짜] 크기의 이중리스트 생성
                        if z == 0:
                            try:
                                sum_fake_list5[i][z] = daejeon[i]
                            except:
                                continue
                        elif z == 1:
                            try:
                                sum_fake_list5[i][z] = '1kg'
                            except:
                                continue

                        elif z == 2:
                            try:
                                sum_fake_list5[i][z] = '대전'
                            except:
                                continue

                        elif z == 3:
                            try:
                                sum_fake_list5[i][z] = lost_date[i]
                            except:
                                continue

        sum_pro_data1 = pro_duct1 + sum_fake_list1
        sum_pro_data2 = pro_duct2 + sum_fake_list2
        sum_pro_data3 = pro_duct3 + sum_fake_list3
        sum_pro_data4 = pro_duct4 + sum_fake_list4
        sum_pro_data5 = pro_duct5 + sum_fake_list5
        loca_data18 = sum_pro_data1 + sum_pro_data2 + sum_pro_data3 + sum_pro_data4 + sum_pro_data5

loc1 = loca_data1 + loca_data11
loc2 = loca_data2 + loca_data12
loc3 = loca_data3 + loca_data13
loc4 = loca_data4 + loca_data14
loc5 = loca_data5 + loca_data15
loc6 = loca_data6 + loca_data16
loc7 = loca_data7 + loca_data17
loc8 = loca_data8 + loca_data18
checkall_data = loc1 + loc2 + loc3 + loc4 + loc5 + loc6 + loc7 + loc8
print("데이터크기 :", len(checkall_data))
LOCATION = r"C:\\instantclient-basic-windows.x64-19.11.0.0.0dbru\\instantclient_19_11"  # 여러분들의 instantclient 가 있는 경로, \표시는 두개로 써줘야함

os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"]
conn = cx_Oracle.connect("waap", "12345", "localhost:1521/xe")  # 여러분들의 접속 이름, 비밀번호, 호스트이름, 포트, sid

cursor = conn.cursor()

print("평균가격 : ", str(avg_plus_avg))
cursor.execute("delete PRO")
conn.commit()
k = 0

for pro in range(len(kindcode)):

    if kindcode[pro] == '쌀':
        for k in range(len(loc1)):
            sql = "insert into pro values(PRO_CODE.NEXTVAL,:PRO_AVER_COST,:PRO_UNIT, :PRO_AREA, to_date(:WEATHER_PERIOD,'YYYY-MM-DD'),:PRO_SUB_CODE,:TEMP_UPDOWN)"
            data = [loc1[k][0], loc1[k][1], loc1[k][2], loc1[k][3], 1, 0]

            cursor.execute(sql, data)
            conn.commit()

    elif kindcode[pro] == '콩':
        for k in range(len(loc1)):
            sql = "insert into pro values(PRO_CODE.NEXTVAL,:PRO_AVER_COST,:PRO_UNIT, :PRO_AREA, to_date(:WEATHER_PERIOD,'YYYY-MM-DD'),:PRO_SUB_CODE,:TEMP_UPDOWN)"
            data = [loc2[k][0], loc2[k][1], loc2[k][2], loc2[k][3], 2, 0]

            cursor.execute(sql, data)
            conn.commit()
    elif kindcode[pro] == '팥':
        for k in range(len(loc1)):
            sql = "insert into pro values(PRO_CODE.NEXTVAL,:PRO_AVER_COST,:PRO_UNIT, :PRO_AREA, to_date(:WEATHER_PERIOD,'YYYY-MM-DD'),:PRO_SUB_CODE,:TEMP_UPDOWN)"
            data = [loc3[k][0], loc3[k][1], loc3[k][2], loc3[k][3], 3, 0]

            cursor.execute(sql, data)
            conn.commit()

    elif kindcode[pro] == '고구마':
        for k in range(len(loc1)):
            sql = "insert into pro values(PRO_CODE.NEXTVAL,:PRO_AVER_COST,:PRO_UNIT, :PRO_AREA, to_date(:WEATHER_PERIOD,'YYYY-MM-DD'),:PRO_SUB_CODE,:TEMP_UPDOWN)"
            data = [loc4[k][0], loc4[k][1], loc4[k][2], loc4[k][3], 4, 0]

            cursor.execute(sql, data)
            conn.commit()

    elif kindcode[pro] == '감자':
        for k in range(len(loc1)):
            sql = "insert into pro values(PRO_CODE.NEXTVAL,:PRO_AVER_COST,:PRO_UNIT, :PRO_AREA, to_date(:WEATHER_PERIOD,'YYYY-MM-DD'),:PRO_SUB_CODE,:TEMP_UPDOWN)"
            data = [loc5[k][0], loc5[k][1], loc5[k][2], loc5[k][3], 5, 0]

            cursor.execute(sql, data)
            conn.commit()


    elif kindcode[pro] == '시금치':
        for k in range(len(loc1)):
            sql = "insert into pro values(PRO_CODE.NEXTVAL,:PRO_AVER_COST,:PRO_UNIT, :PRO_AREA, to_date(:WEATHER_PERIOD,'YYYY-MM-DD'),:PRO_SUB_CODE,:TEMP_UPDOWN)"
            data = [loc6[k][0], loc6[k][1], loc6[k][2], loc6[k][3], 6, 0]

            cursor.execute(sql, data)
            conn.commit()
    elif kindcode[pro] == '열무':
        for k in range(len(loc1)):
            sql = "insert into pro values(PRO_CODE.NEXTVAL,:PRO_AVER_COST,:PRO_UNIT, :PRO_AREA, to_date(:WEATHER_PERIOD,'YYYY-MM-DD'),:PRO_SUB_CODE,:TEMP_UPDOWN)"
            data = [loc7[k][0], loc7[k][1], loc7[k][2], loc7[k][3], 7, 0]

            cursor.execute(sql, data)
            conn.commit()

    elif kindcode[pro] == '대파':
        for k in range(len(loc1)):
            sql = "insert into pro values(PRO_CODE.NEXTVAL,:PRO_AVER_COST,:PRO_UNIT, :PRO_AREA, to_date(:WEATHER_PERIOD,'YYYY-MM-DD'),:PRO_SUB_CODE,:TEMP_UPDOWN)"
            data = [loc8[k][0], loc8[k][1], loc8[k][2], loc8[k][3], 8, 0]

            cursor.execute(sql, data)
            conn.commit()

cursor.execute("select * from PRO")
conn.commit()

cursor.close()
conn.close()



쌀
쌀   데이터 정제 시작
콩
콩   데이터 정제 시작
팥
팥   데이터 정제 시작
고구마
고구마   데이터 정제 시작
감자
감자   데이터 정제 시작
시금치
시금치   데이터 정제 시작
열무
열무   데이터 정제 시작
대파
대파   데이터 정제 시작
쌀
쌀   데이터 정제 시작
콩
콩   데이터 정제 시작
팥
팥   데이터 정제 시작
고구마
고구마   데이터 정제 시작
감자
감자   데이터 정제 시작
시금치
시금치   데이터 정제 시작
열무
열무   데이터 정제 시작
대파
대파   데이터 정제 시작
데이터크기 : 14720
평균가격 :  2956
